<a href="https://colab.research.google.com/github/SM-Learning/advanced-rag-techniques/blob/main/NER_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers torch inflect teradatasql pyspark GLiNER

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q --upgrade GLiNER


In [ ]:
from gliner import GLiNER

In [ ]:
import datetime
import logging
import re
from itertools import chain
import torch
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from inflect import engine
import pandas as pd
from gliner import GLiNER
#from pyspark.sql.window import Window

# Initialize inflect engine
p = engine()

# Initialize logging
# Configure logging with datetime
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)


In [ ]:
# Initialize Spark session
#spark = SparkSession.builder.appName("NER_Processing").getOrCreate()

spark = SparkSession.builder \
       .appName("NER_Processing") \
       .config("spark.driver.memory", "8g") \
       .config("spark.executor.memory", "8g") \
       .getOrCreate()

In [ ]:
# Generate more complete sample data
sample_data = [
    {
        "company_id": "1",
        "es_ecid": "EC001",
        "description": "Google is an American multinational technology company focusing on AI, cloud computing, and online advertising.",
        "industry_primary": "Technology",
        "industries": "Software, Internet",
        "sub_industries": "Search Engine, Cloud Computing",
        "top3_industries": "Technology, Advertising, AI"
    },
    {
        "company_id": "2",
        "es_ecid": "EC002",
        "description": "Microsoft Corporation develops software and provides cloud services.",
        "industry_primary": "Technology",
        "industries": "Software",
        "sub_industries": "Operating Systems, Cloud Computing",
        "top3_industries": "Software, Cloud, Enterprise"
    }
]

In [ ]:
# Create DataFrame with all required columns
spark_df = spark.createDataFrame(sample_data)

In [ ]:
# Initialize logging
#logging.basicConfig(level=logging.INFO)

# Batch sizes
batch_size = 50
#batch_size_db = 100
table_name = "xyz"

# GLiNER model initialization
try:
    model_name = "knowledgator/gliner-multitask-large-v0.5"
    model = GLiNER.from_pretrained(model_name)
    model_name_2 = "EmergentMethods/gliner_large_news-v2.1"
    model_2 = GLiNER.from_pretrained(model_name_2)
except Exception as e:
    logger.error(f"Error loading models: {e}")
    raise

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/14.5k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

gliner_multitask_performance.png:   0%|          | 0.00/76.8k [00:00<?, ?B/s]

...

assets%2Fcountries_distribution.png:   0%|          | 0.00/398k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Labels for NER
labels = {
    "year started": None,
    "product": None,
    "brand": None,
    "business categories": None,
    "business sub categories": None,
    "industry": None,
    "business services": None,
    "offer": None,
}

# Schema definition
ner_schema = StructType([
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

In [ ]:
def clean_entity(entity):
    try:
        if entity['label'] == 'year started':
            match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
            entity['text'] = match.group(0) if match else ''
        elif entity['label'] not in ['brand']:
            entity['text'] = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
            entity['text'] = p.singular_noun(entity['text']) or entity['text']
    except Exception as e:
        logger.error(f"Error cleaning entity: {e}")
    return entity

In [ ]:
def extract_ner_features(text):
    if not text or len(str(text)) < 50:
        logger.warning(f"Skipping text with length < 50 characters")
        return {}

    information = {}
    entity_sets = {}

    try:
        with torch.no_grad():
            entities = model.predict_entities(text, labels, threshold=0.54)
            entities_2 = model_2.predict_entities(text, labels, threshold=0.7)
            entities = sorted(
                (clean_entity(entity) for entity in chain(entities, entities_2)),
                key=lambda k: (k['label'], -k['score'])
            )

            for entity in entities:
                label = entity["label"]
                text = entity["text"]

                if label == 'brand':
                    text = text.replace('brands', '').replace('brand', '').strip()
                elif label == 'industry':
                    text = text.lower().replace('industry', '').replace('industries', '').strip()
                elif label == 'product':
                    text = text.lower().replace('products', '').replace('product', '').strip()

                if label == "year started" and "year started" not in information:
                    information["year started"] = text

                # Combine entities with delimiter
                if label not in entity_sets:
                    entity_sets[label] = set()
                entity_sets[label].add(text)

            for field in labels.keys():
                if field in entity_sets:
                    information[field.replace(" ", "_")] = " | ".join(entity_sets[field])

    except Exception as e:
        logger.error(f"Error in NER processing: {e}")

    return information

'''
            # Combine entities with delimiter
            if "product" in entity_sets:
                information["product"] = " | ".join(entity_sets["product"])
            if "brand" in entity_sets:
                information["brand"] = " | ".join(entity_sets["brand"])
            if "business categories" in entity_sets:
                information["business_categories"] = " | ".join(entity_sets["business categories"])
            if "business sub categories" in entity_sets:
                information["business_sub_categories"] = " | ".join(entity_sets["business sub categories"])
            if "industry" in entity_sets:
                information["industry"] = " | ".join(entity_sets["industry"])
            if "business services" in entity_sets:
                information["business_services"] = " | ".join(entity_sets["business services"])
            if "offer" in entity_sets:
                information["offer"] = " | ".join(entity_sets["offer"])
'''

In [ ]:
def process_data():
    # Create DataFrame
    spark_df = spark.createDataFrame(sample_data)

    # Filter descriptions
    spark_df = spark_df.filter(F.length("description") >= 50)

    # Register UDF
    extract_ner_features_udf = F.udf(extract_ner_features, ner_schema)

    # Process data
    processed_df = spark_df.withColumn(
        "ner_features",
        extract_ner_features_udf("description")
    ).select("*", "ner_features.*").drop("ner_features")

    # Add missing columns and handle null values
    required_columns = [
        'company_id', 'es_ecid', 'description', 'industry_primary', 'industries',
        'sub_industries', 'top3_industries', 'year_started', 'product', 'brand',
        'industry', 'business_categories', 'business_sub_categories', 'business_services', 'offer'
    ]

    for column in required_columns:
        if column not in processed_df.columns:
            processed_df = processed_df.withColumn(column, F.lit(""))

    # Cleanse the NER responses before loading/printing
    final_df = (processed_df
        .select(required_columns)
        .na.fill("")
        .replace({"nan": "", "None": ""})
    )

    return final_df

In [ ]:
# Main execution
if __name__ == "__main__":
    logger.info(f"Starting NER processing at: {datetime.datetime.now()}")

    try:
        result_df = process_data()

        # Convert to Pandas for display
        pandas_df = result_df.toPandas()

        logger.info("\nFinal DataFrame:")
        print(pandas_df.to_string())

    except Exception as e:
        logger.error(f"Error in main execution: {e}")

    logger.info(f"Processing completed at: {datetime.datetime.now()}")

In [ ]:
# Debug output
print("\nProcessed DataFrame Schema:")
final_df.printSchema()

In [ ]:
print("\nProcessed DataFrame Content:")
final_df.show(truncate=False)

In [ ]:
print("End time: ", datetime.datetime.now())

End time:  2025-02-17 05:11:55.737103


In [ ]:
def upload_to_teradata(df, table_name, batch_size):
    """
    Upload DataFrame to Teradata in batches
    """
    try:
        with teradatasql.connect(
            host=hostname,
            user=user_id,
            password=pwd,
            logmech=logmech
        ) as con:
            cursor = con.cursor()

            # Define insert SQL statement
            insert_sql = f"""
            INSERT INTO {table_name} (
                zi_c_company_id, zi_es_ecid, zi_c_description,
                zi_industry_primary, es_ecid, description,
                industry_primary, industries, sub_industries,
                top3_industries
            ) VALUES (?,?,?,?,?,?,?,?,?,?)
            """

            # Convert DataFrame to list of tuples
            data = [tuple(x) for x in df.values]

            # Process in batches
            for i in range(0, len(data), batch_size):
                try:
                    batch = data[i:i + batch_size]
                    cursor.executemany(insert_sql, batch)
                    logger.info(f"Batch {i // batch_size + 1} uploaded at: {datetime.datetime.now()}")

                except Exception as e:
                    logger.error(f"Error uploading batch {i // batch_size + 1}: {e}")
                    logger.error(f"Offending batch: {batch}")
                    continue

    except Exception as e:
        logger.error(f"Database connection error: {e}")
        raise

def process_and_upload_data(spark_df):
    """
    Process data and upload to Teradata
    """
    logger.info(f"Starting processing at: {datetime.datetime.now()}")

    try:
        # Convert Spark DataFrame to Pandas
        df = spark_df.toPandas()

        # Process in batches
        for i in range(0, len(df), batch_size):
            # Extract NER features
            df.loc[i:i + batch_size - 1, 'entities'] = \
                df.loc[i:i + batch_size - 1, 'description'].apply(extract_ner_features).values

            logger.info(f"Batch {i // batch_size + 1} NER processing completed at: {datetime.datetime.now()}")

            # Extract the processed batch
            df_batch = df.loc[i:i + batch_size - 1].copy()

            # Expand entities column
            df_batch = pd.concat([
                df_batch.drop('entities', axis=1),
                df_batch['entities'].apply(pd.Series)
            ], axis=1)

            # Ensure all required columns exist
            required_columns = [
                'company_id', 'es_ecid', 'description', 'industry_primary',
                'industries', 'sub_industries', 'top3_industries',
                'year_started', 'product', 'brand', 'industry',
                'business_categories', 'business_sub_categories',
                'business_services', 'offer'
            ]

            # Add missing columns
            for column in required_columns:
                if column not in df_batch.columns:
                    df_batch[column] = ''

            # Select and order columns
            df_batch = df_batch[required_columns]

            # Clean up data
            df_batch = df_batch.astype(str)
            df_batch = df_batch.replace({'nan': '', 'None': ''})
            df_batch = df_batch.fillna('')

            # Upload batch to Teradata
            try:
                upload_to_teradata(df_batch, table_name, batch_size)
            except Exception as e:
                logger.error(f"Error uploading batch: {e}")
                continue

        logger.info(f"Processing completed at: {datetime.datetime.now()}")
        return df  # Return final DataFrame

    except Exception as e:
        logger.error(f"Error in process_and_upload_data: {e}")
        raise

# Main execution
if __name__ == "__main__":
    logger.info(f"Starting NER processing and Teradata upload at: {datetime.datetime.now()}")

    try:
        # Create Spark DataFrame from sample data
        spark = SparkSession.builder.appName("NER_Processing").getOrCreate()
        spark_df = spark.createDataFrame(sample_data)

        # Process data and upload to Teradata
        result_df = process_and_upload_data(spark_df)

        # Display final results
        logger.info("\nFinal DataFrame:")
        print(result_df.to_string())

    except Exception as e:
        logger.error(f"Error in main execution: {e}")

    logger.info(f"Process completed at: {datetime.datetime.now()}")


# Version #2 - Updated for spark cluster
```
# Version #2 - updated for spark cluster
```



In [ ]:
import torch
from pyspark.sql import SparkSession
import pandas as pd
import datetime
import logging
import teradatasql
import re
import inflect
from itertools import chain

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize inflect engine
p = inflect.engine()

# Create Spark DataFrame from pandas
spark_df = spark.createDataFrame(df).repartition(128)

In [ ]:
# Broadcast GLiNER models
def load_and_broadcast_models():
    model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
    model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")
    return spark.sparkContext.broadcast((model1, model2))

models_broadcast = load_and_broadcast_models()

labels = {
    "headquarter": ["head office", "HQ"],
    "company name": ["company", "provider", "organization", "employer", "firm", "business"],
    "product": None,
    "brand": None
}

def clean_entity(entity):
    try:
        if entity['label'] == 'year started':
            match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
            entity['text'] = match.group(0) if match else ''
        elif entity['label'] not in ['brand']:
            entity['text'] = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
            entity['text'] = p.singular_noun(entity['text']) or entity['text']
    except:
        pass
    return entity

def extract_ner_features(text):
    information = {}
    entity_sets = {}

    # Get models from broadcast variable
    model, model_2 = broadcast_models.value

    with torch.no_grad():
        entities = model.predict_entities(text, labels, threshold=0.54)
        entities_2 = model_2.predict_entities(text, labels, threshold=0.7)
        entities = sorted((clean_entity(entity) for entity in chain(entities, entities_2)),
                        key=lambda k: (k['label'], -k['score']))

    for entity in entities:
        label = entity["label"]
        text = entity["text"]

        if label == "brand":
            text = text.replace('brands', '').replace('brand', '').strip()
        elif label == "industry":
            text = text.lower().replace('industry', '').strip()
        elif label == "product":
            text = text.lower().replace('products', '').replace('product', '').strip()

        if label not in entity_sets:
            entity_sets[label] = set()
        entity_sets[label].add(text)

        if (label == "year started") and "year started" not in information:
            information["year started"] = text

    if "product" in entity_sets:
        information["product"] = " | ".join(entity_sets["product"])
    if "brand" in entity_sets:
        information["brand"] = " | ".join(entity_sets["brand"])
    if "business categories" in entity_sets:
        information["business_categories"] = " | ".join(entity_sets["business categories"])
    if "business sub categories" in entity_sets:
        information["business_sub_categories"] = " | ".join(entity_sets["business sub categories"])
    if "industry" in entity_sets:
        information["industry"] = " | ".join(entity_sets["industry"])
    if "business services" in entity_sets:
        information["business_services"] = " | ".join(entity_sets["business services"])
    if "offer" in entity_sets:
        information["offer"] = " | ".join(entity_sets["offer"])

    return information

def upload_to_teradata(df, table_name, batch_size_db):
    with teradatasql.connect(host="hostname", user="user_id", password="pwd", logmech="logmech") as con:
        insert_sql = f"""
        INSERT INTO {table_name} (
            zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
            industries, sub_industries, top3_industries, year_started, product,
            brand, industry, business_categories, business_sub_categories,
            business_services, offer
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        data = [tuple(x) for x in df.values]

        for i in range(0, len(data), batch_size_db):
            try:
                batch = data[i:i + batch_size_db]
                cursor = con.cursor()
                cursor.executemany(insert_sql, batch)
                print(f"batch {i // batch_size_db + 1} uploaded at: ", datetime.datetime.now())
            except Exception as e:
                logging.error(f"Error uploading batch {i // batch_size_db + 1}: {e}")
                logging.error(f"Offending batch: {batch}")
                continue

def main():
    batch_size = 50
    batch_size_db = 100
    table_name = "xyz"

    print("Start time: ", datetime.datetime.now())

    for i in range(0, len(df), batch_size):
        # Process batch
        df.loc[i:i + batch_size - 1, 'entities'] = df.loc[i:i + batch_size - 1, 'description'].apply(
            extract_ner_features).values
        print(f"\nbatch {i // batch_size + 1} completed at: ", datetime.datetime.now())

        # Extract the processed batch
        df_batch = df.loc[i:i + batch_size - 1].copy()

        # Combine processed results
        df_batch = pd.concat([df_batch.drop('entities', axis=1),
                            df_batch['entities'].apply(pd.Series)], axis=1)

        required_columns = [
            'company_id', 'es_ecid', 'description', 'industry_primary',
            'industries', 'sub_industries', 'top3_industries', 'year_started',
            'product', 'brand', 'industry', 'business_categories',
            'business_sub_categories', 'business_services', 'offer'
        ]

        # Add missing columns
        for column in required_columns:
            if column not in df_batch.columns:
                df_batch[column] = ''

        # Select and order columns
        df_batch = df_batch[required_columns]

        # Clean up data
        df_batch = df_batch.astype(str).replace({'nan': '', 'None': ''})
        df_batch = df_batch.fillna('')

        # Upload batch
        try:
            upload_to_teradata(df_batch, table_name, batch_size_db)
        except Exception as e:
            logging.error(f"Error uploading batch: {e}")

    print("End time: ", datetime.datetime.now())


In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StructType, StructField, StringType
import functools

# Define the schema for the output
ner_schema = StructType([
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

# Cache for models at worker level
_model_cache = {}

def get_or_load_models():
    """Load models once per worker"""
    worker_id = hash(threading.current_thread().ident)
    if worker_id not in _model_cache:
        model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
        model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")
        _model_cache[worker_id] = (model1, model2)
    return _model_cache[worker_id]

@pandas_udf(ner_schema)
def extract_ner_features_batch(texts: pd.Series) -> pd.DataFrame:
    """Process a batch of texts using cached models"""
    # Load models once per worker
    model1, model2 = get_or_load_models()

    results = []
    with torch.no_grad():
        for text in texts:
            entities = model1.predict_entities(text, labels, threshold=0.54)
            entities_2 = model2.predict_entities(text, labels, threshold=0.7)
            entities = sorted(
                (clean_entity(entity) for entity in chain(entities, entities_2)),
                key=lambda k: (k['label'], -k['score'])
            )

            # Process entities and create result dictionary
            information = process_entities(entities)
            results.append(information)

    return pd.DataFrame(results)

def process_entities(entities):
    """Process entities and return formatted information"""
    information = {}
    entity_sets = {}

    for entity in entities:
        label = entity["label"]
        text = entity["text"]

        if label == "brand":
            text = text.replace('brands', '').replace('brand', '').strip()
        elif label == "industry":
            text = text.lower().replace('industry', '').strip()
        elif label == "product":
            text = text.lower().replace('products', '').replace('product', '').strip()

        if label not in entity_sets:
            entity_sets[label] = set()
        entity_sets[label].add(text)

        if (label == "year started") and "year started" not in information:
            information["year started"] = text

    # Fill in all possible fields
    fields = ["product", "brand", "business_categories", "business_sub_categories",
              "year_started", "industry", "business_services", "offer"]

    for field in fields:
        field_key = field.replace("_", " ")
        information[field] = " | ".join(entity_sets.get(field_key, [])) if field_key in entity_sets else ""

    return information

# Main processing logic
def process_dataframe(spark_df, batch_size=1000):
    """Process the DataFrame in batches"""
    return (spark_df
            .repartition(128)  # Adjust based on cluster size
            .withColumn("ner_features", extract_ner_features_batch("description"))
            .select("*", "ner_features.*")
            .drop("ner_features"))

# Usage example
try:
    processed_df = process_dataframe(spark_df)

    # Process in batches for Teradata upload
    pandas_df = processed_df.toPandas()
    for i in range(0, len(pandas_df), batch_size_db):
        batch = pandas_df.iloc[i:i + batch_size_db]
        upload_to_teradata(batch, table_name, batch_size_db)

except Exception as e:
    logging.error(f"Error in processing: {e}")

# Version 3

In [ ]:
import torch
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import logging
import re
from itertools import chain
import inflect
import teradatasql
import pandas as pd
from pyspark.sql.functions import pandas_udf
from typing import Iterator

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Spark session
spark = SparkSession.builder \
    .appName("NER Processing with Spark") \
    .getOrCreate()

# Define schema for NER output
ner_schema = StructType([
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

# Load and broadcast models
def load_and_broadcast_models():
    model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
    model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")
    return spark.sparkContext.broadcast((model1, model2))

# Broadcast variables
broadcast_models = load_and_broadcast_models()
p = inflect.engine()

# Broadcast labels dictionary
labels = spark.sparkContext.broadcast({
    "headquarter": ["head office", "HQ"],
    "company name": ["company", "provider", "organization", "employer", "firm", "business"],
    "product": None,
    "brand": None
})

@pandas_udf(ner_schema)
def extract_ner_features_udf(texts: Iterator[pd.Series]) -> Iterator[pd.Series]:
    # Get models from broadcast variable
    model, model_2 = broadcast_models.value
    labels_dict = labels.value

    def process_single_text(text):
        information = {}
        entity_sets = {}

        with torch.no_grad():
            entities = model.predict_entities(text, labels_dict, threshold=0.54)
            entities_2 = model_2.predict_entities(text, labels_dict, threshold=0.7)
            entities = sorted((clean_entity(entity) for entity in chain(entities, entities_2)),
                            key=lambda k: (k['label'], -k['score']))

        for entity in entities:
            label = entity["label"]
            text = entity["text"]

            if label == "brand":
                text = text.replace('brands', '').replace('brand', '').strip()
            elif label == "industry":
                text = text.lower().replace('industry', '').strip()
            elif label == "product":
                text = text.lower().replace('products', '').replace('product', '').strip()

            if label not in entity_sets:
                entity_sets[label] = set()
            entity_sets[label].add(text)

        # Fill in all fields
        result = {
            "product": " | ".join(entity_sets.get("product", [])),
            "brand": " | ".join(entity_sets.get("brand", [])),
            "business_categories": " | ".join(entity_sets.get("business categories", [])),
            "business_sub_categories": " | ".join(entity_sets.get("business sub categories", [])),
            "year_started": information.get("year_started", ""),
            "industry": " | ".join(entity_sets.get("industry", [])),
            "business_services": " | ".join(entity_sets.get("business services", [])),
            "offer": " | ".join(entity_sets.get("offer", []))
        }
        return pd.Series(result)

    for batch in texts:
        yield batch.apply(process_single_text)

def upload_to_teradata(df, table_name, batch_size_db):
    """Upload data to Teradata using teradatasql."""
    with teradatasql.connect(host="hostname", user="user_id", password="password", logmech="logmech") as con:
        insert_sql = f"""
        INSERT INTO {table_name} (
            zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
            industries, sub_industries, top3_industries, year_started, product,
            brand, industry, business_categories, business_sub_categories,
            business_services, offer
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        data = [tuple(x) for x in df.values]

        for i in range(0, len(data), batch_size_db):
            try:
                batch = data[i:i + batch_size_db]
                cursor = con.cursor()
                cursor.executemany(insert_sql, batch)
                logging.info(f"Batch {i // batch_size_db + 1} uploaded successfully.")
            except Exception as e:
                logging.error(f"Error uploading batch {i // batch_size_db + 1}: {e}")
                logging.error(f"Offending batch: {batch}")
                continue

def process_and_display_dataframe(spark_df, table_name, batch_size_db=100):
    """Process the Spark DataFrame with NER extraction and display results."""
    # Repartition for better distribution
    spark_df = spark_df.repartition(128)

    # Apply NER extraction
    processed_df = spark_df.withColumn(
        "ner_features",
        extract_ner_features_udf("description")
    ).select(
        "*",
        "ner_features.*"
    ).drop("ner_features")

    # Convert Spark DataFrame to Pandas for display
    pandas_df = processed_df.toPandas()

    # Print each batch DataFrame
    for i in range(0, len(pandas_df), batch_size_db):
        batch = pandas_df.iloc[i:i + batch_size_db]
        print("\nBatch DataFrame:")
        print(batch)

        # Commented out the upload function
        # upload_to_teradata(batch, table_name, batch_size_db)

def main():
    # Example Spark DataFrame
    sample_data = [
        {"company_id": "1", "description": "Google is a multinational technology company."},
        {"company_id": "2", "description": "Microsoft develops software and provides cloud services."}
    ]
    spark_df = spark.createDataFrame(sample_data)

    # Process and display results
    table_name = "xyz"
    process_and_display_dataframe(spark_df, table_name)

if __name__ == "__main__":
    main()

In [ ]:
## iterator based UDF code.

def extract_ner_features(text):
    """
    Extract NER features for a single text input.
    """
    information = {}
    entity_sets = {}

    # Get models from broadcast variable
    model1, model2 = broadcast_models.value
    labels_dict = labels.value

    with torch.no_grad():
        # Predict entities using both models
        entities = model1.predict_entities(text, labels_dict, threshold=0.54)
        entities_2 = model2.predict_entities(text, labels_dict, threshold=0.7)
        entities = sorted(
            (clean_entity(entity) for entity in chain(entities, entities_2)),
            key=lambda k: (k['label'], -k['score'])
        )

    # Process entities and group them by label
    for entity in entities:
        label = entity["label"]
        text = entity["text"]

        # Clean specific entity types
        if label == "brand":
            text = text.replace('brands', '').replace('brand', '').strip()
        elif label == "industry":
            text = text.lower().replace('industry', '').strip()
        elif label == "product":
            text = text.lower().replace('products', '').replace('product', '').strip()

        # Add to entity sets
        if label not in entity_sets:
            entity_sets[label] = set()
        entity_sets[label].add(text)

        # Handle "year started" separately
        if label == "year started" and "year started" not in information:
            information["year_started"] = text

    # Combine entities into a single dictionary
    fields = [
        "year_started", "product", "brand", "business_categories",
        "business_sub_categories", "industry", "business_services", "offer"
    ]
    for field in fields:
        information[field] = " | ".join(entity_sets.get(field, []))

    return information

In [ ]:
@pandas_udf(ner_schema)
def extract_ner_features_udf(batch_iter: Iterator[pd.Series]) -> Iterator[pd.DataFrame]:
    """
    Iterator-based UDF to process batches of text and extract NER features.
    """
    for batch in batch_iter:
        # Apply the extract_ner_features function to each text in the batch
        results = batch.apply(extract_ner_features)
        # Convert the results (list of dictionaries) into a Pandas DataFrame
        yield pd.DataFrame(results.tolist())

**Map Partitions**

better over UDFs for larger dataset or larger models.

In [ ]:
import torch
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import logging
import re
from itertools import chain
import inflect
import teradatasql
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Spark session
spark = SparkSession.builder \
    .appName("NER Processing with mapPartitions") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

# Define schema for the processed DataFrame
processed_schema = StructType([
    StructField("company_id", StringType(), True),
    StructField("es_ecid", StringType(), True),
    StructField("description", StringType(), True),
    StructField("industry_primary", StringType(), True),
    StructField("industries", StringType(), True),
    StructField("sub_industries", StringType(), True),
    StructField("top3_industries", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

# Pre-processing: Clean entity function
def clean_entity(entity):
    """Clean and process entity text."""
    try:
        if entity['label'] == 'year started':
            match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
            entity['text'] = match.group(0) if match else ''
        elif entity['label'] not in ['brand']:
            entity['text'] = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
            entity['text'] = inflect.engine().singular_noun(entity['text']) or entity['text']
    except Exception as e:
        logging.error(f"Error cleaning entity: {e}")
    return entity

# Post-processing: Add missing fields and clean up
def post_process_row(row):
    """Ensure all required fields are present and clean up the row."""
    required_fields = [
        "year_started", "product", "brand", "industry", "business_categories",
        "business_sub_categories", "business_services", "offer"
    ]
    for field in required_fields:
        if field not in row or row[field] is None:
            row[field] = ""
    return row

# Partition processing function
def extract_ner_entities(iterator):
    """
    Process a partition of data and apply NER extraction.
    """
    # Load models for the partition
    model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
    model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")
    labels_dict = {
        "headquarter": ["head office", "HQ"],
        "company name": ["company", "provider", "organization", "employer", "firm", "business"],
        "product": None,
        "brand": None
    }

    # Process each row in the partition
    for row in iterator:
        try:
            # Extract NER features
            with torch.no_grad():
                entities = model1.predict_entities(row.description, labels_dict, threshold=0.54)
                entities_2 = model2.predict_entities(row.description, labels_dict, threshold=0.7)
                entities = sorted(
                    (clean_entity(entity) for entity in chain(entities, entities_2)),
                    key=lambda k: (k['label'], -k['score'])
                )

            # Combine entities into a single dictionary
            ner_features = {}
            entity_sets = {}
            for entity in entities:
                label = entity["label"]
                text = entity["text"]

                if label not in entity_sets:
                    entity_sets[label] = set()
                entity_sets[label].add(text)

                if label == "year started" and "year started" not in ner_features:
                    ner_features["year_started"] = text

            # Fill in all fields
            fields = [
                "year_started", "product", "brand", "industry", "business_categories",
                "business_sub_categories", "business_services", "offer"
            ]
            for field in fields:
                ner_features[field] = " | ".join(entity_sets.get(field, []))

            # Post-process the row
            ner_features = post_process_row(ner_features)

            # Yield the processed row
            yield (
                row.company_id,
                row.es_ecid,
                row.description,
                row.industry_primary,
                row.industries,
                row.sub_industries,
                row.top3_industries,
                ner_features["year_started"],
                ner_features["product"],
                ner_features["brand"],
                ner_features["industry"],
                ner_features["business_categories"],
                ner_features["business_sub_categories"],
                ner_features["business_services"],
                ner_features["offer"]
            )
        except Exception as e:
            logging.error(f"Error processing row: {e}")
            continue

# Upload to Teradata
def upload_to_teradata(df, table_name, batch_size_db):
    """Upload data to Teradata using teradatasql."""
    with teradatasql.connect(host="hostname", user="user_id", password="password", logmech="logmech") as con:
        insert_sql = f"""
        INSERT INTO {table_name} (
            company_id, es_ecid, description, industry_primary,
            industries, sub_industries, top3_industries, year_started, product,
            brand, industry, business_categories, business_sub_categories,
            business_services, offer
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        data = [tuple(x) for x in df.values]

        for i in range(0, len(data), batch_size_db):
            try:
                batch = data[i:i + batch_size_db]
                cursor = con.cursor()
                cursor.executemany(insert_sql, batch)
                logging.info(f"Batch {i // batch_size_db + 1} uploaded successfully.")
            except Exception as e:
                logging.error(f"Error uploading batch {i // batch_size_db + 1}: {e}")
                logging.error(f"Offending batch: {batch}")
                continue

# Main processing function
def process_and_load_dataframe(spark_df, table_name, batch_size_db=100):
    """
    Process the Spark DataFrame with NER extraction using mapPartitions and upload results to Teradata.
    """
    # Repartition for better distribution
    spark_df = spark_df.repartition(64)  # Adjust based on dataset size and cluster configuration
    logging.info(f"Repartitioned DataFrame into {spark_df.rdd.getNumPartitions()} partitions.")

    # Convert DataFrame to RDD and apply mapPartitions
    processed_rdd = spark_df.rdd.mapPartitions(extract_ner_entities)

    # Convert the RDD back to a DataFrame
    processed_df = spark.createDataFrame(processed_rdd, schema=processed_schema)

    # Log metrics after processing
    logging.info(f"Processed DataFrame contains {processed_df.count()} records after NER extraction.")
    logging.info(f"Number of partitions after processing: {processed_df.rdd.getNumPartitions()}")

    # Convert Spark DataFrame to Pandas for batch upload
    pandas_df = processed_df.toPandas()

    # Upload each batch to Teradata
    for i in range(0, len(pandas_df), batch_size_db):
        batch = pandas_df.iloc[i:i + batch_size_db]
        logging.info(f"Uploading batch {i // batch_size_db + 1} with {len(batch)} records to Teradata.")
        upload_to_teradata(batch, table_name, batch_size_db)

# Save file to CSV and later upload the database

In [ ]:
import os
import logging
import pandas as pd
import teradatasql
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Path for the CSV file
csv_file_path = "/data/ner.csv"

def save_spark_df_to_csv(spark_df, csv_path):
    """
    Save Spark DataFrame to CSV file
    """
    try:
        # Start timer
        start_time = datetime.now()
        logger.info(f"Starting to write Spark DataFrame to CSV at {csv_path}")

        # Get number of records
        record_count = spark_df.count()
        logger.info(f"Writing {record_count} records to CSV")

        # Write DataFrame to CSV
        # coalesce(1) to write to a single file - remove for very large datasets
        # For large datasets, you may want to use multiple partitions
        spark_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(csv_path)

        # Find the single part file and rename it to ner.csv
        part_file = [f for f in os.listdir(csv_dir_path) if f.startswith("part-")][0]
        full_part_file_path = os.path.join(csv_dir_path, part_file)
        final_csv_path = os.path.join(csv_dir_path, "ner.csv")
        os.rename(full_part_file_path, final_csv_path)

        # Calculate elapsed time
        elapsed_time = datetime.now() - start_time
        logger.info(f"Successfully wrote Spark DataFrame to CSV in {elapsed_time}")

        return True
    except Exception as e:
        logger.error(f"Error writing Spark DataFrame to CSV: {str(e)}")
        return False

def upload_csv_to_teradata(csv_path, table_name, batch_size=10000):
    """
    Upload CSV file to Teradata table
    """
    # Teradata connection parameters
    teradata_params = {
        "host": "your_teradata_host",
        "user": "your_username",
        "password": "your_password",
        "logmech": "LDAP",  # or other authentication mechanism
        "database": "your_database"
    }

    try:
        # Start timer
        start_time = datetime.now()
        logger.info(f"Starting to upload CSV to Teradata table {table_name}")

        # Read CSV file in chunks to avoid memory issues
        # Use chunksize parameter to control memory usage
        csv_reader = pd.read_csv(csv_path, chunksize=batch_size)

        # Connect to Teradata
        with teradatasql.connect(
            host=teradata_params["host"],
            user=teradata_params["user"],
            password=teradata_params["password"],
            logmech=teradata_params["logmech"],
            database=teradata_params["database"]
        ) as conn:

            # Get cursor
            cursor = conn.cursor()

            # Process each chunk
            total_rows = 0
            chunk_count = 0

            for chunk in csv_reader:
                chunk_count += 1
                chunk_start_time = datetime.now()
                chunk_size = len(chunk)
                total_rows += chunk_size

                logger.info(f"Processing chunk {chunk_count} with {chunk_size} rows")

                # Prepare column names and placeholders for SQL
                columns = ", ".join(chunk.columns)
                placeholders = ", ".join([f":{i}" for i in range(1, len(chunk.columns) + 1)])

                # Prepare SQL statement
                sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

                # Execute batch insert
                batch_data = []
                for _, row in chunk.iterrows():
                    batch_data.append(tuple(row))

                cursor.executemany(sql, batch_data)
                conn.commit()

                chunk_elapsed_time = datetime.now() - chunk_start_time
                logger.info(f"Chunk {chunk_count} uploaded in {chunk_elapsed_time}")

            # Calculate elapsed time
            elapsed_time = datetime.now() - start_time
            logger.info(f"Successfully uploaded {total_rows} rows to Teradata in {elapsed_time}")

            return True
    except Exception as e:
        logger.error(f"Error uploading CSV to Teradata: {str(e)}")
        return False

def main():
    """
    Main function to execute the pipeline
    """
    try:
        # Assuming spark_df is already defined and contains the processed data

        # Step 1: Save Spark DataFrame to CSV
        logger.info("Step 1: Saving Spark DataFrame to CSV")
        csv_success = save_spark_df_to_csv(spark_df, csv_file_path)

        if not csv_success:
            logger.error("Failed to save Spark DataFrame to CSV. Exiting.")
            return

        # Step 2: Upload CSV to Teradata
        logger.info("Step 2: Uploading CSV to Teradata")
        teradata_table = "your_teradata_table"  # Replace with your table name
        upload_success = upload_csv_to_teradata(csv_file_path, teradata_table)

        if not upload_success:
            logger.error("Failed to upload CSV to Teradata. Exiting.")
            return

        logger.info("Pipeline completed successfully!")

    except Exception as e:
        logger.error(f"Error in main pipeline: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
from pyspark.sql import SparkSession
import logging
from datetime import datetime
import teradatasql
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import BooleanType

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def upload_spark_csv_from_gcs_to_teradata_with_spark(spark, gcs_dir_path, table_name, batch_size=10000):
    """
    Upload Spark-generated CSV from GCS to Teradata table using Spark.
    """
    try:
        # Read the CSV directory (which contains the part file)
        logger.info(f"Reading CSV from {gcs_dir_path}")
        df = spark.read.option("header", "true").csv(gcs_dir_path)

        # Log the schema and count
        logger.info(f"CSV schema: {df.schema}")
        count = df.count()
        logger.info(f"CSV row count: {count}")

        # Define a function to upload a batch of data to Teradata
        def upload_batch_to_teradata(batch_df):
            """
            Upload a batch of data to Teradata.
            """
            try:
                # Teradata connection parameters
                teradata_params = {
                    "host": "your_teradata_host",
                    "user": "your_username",
                    "password": "your_password",
                    "logmech": "LDAP",  # or other authentication mechanism
                    "database": "your_database"
                }

                # Connect to Teradata
                with teradatasql.connect(
                    host=teradata_params["host"],
                    user=teradata_params["user"],
                    password=teradata_params["password"],
                    logmech=teradata_params["logmech"],
                    database=teradata_params["database"]
                ) as conn:
                    # Get cursor
                    cursor = conn.cursor()

                    # Prepare column names and placeholders for SQL
                    columns = ", ".join(batch_df.columns)
                    placeholders = ", ".join([f":{i}" for i in range(1, len(batch_df.columns) + 1)])

                    # Prepare SQL statement
                    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

                    # Execute batch insert
                    batch_data = [tuple(row) for row in batch_df.itertuples(index=False)]
                    cursor.executemany(sql, batch_data)
                    conn.commit()

                    return True
            except Exception as e:
                logger.error(f"Error uploading batch to Teradata: {str(e)}")
                return False

        # Process the DataFrame in batches using foreachPartition
        def process_partition(iterator):
            # Convert iterator to list for easier handling
            rows = list(iterator)
            if not rows:
                return

            # Create a pandas DataFrame from the rows
            batch_df = pd.DataFrame(rows, columns=df.columns)

            # Upload the batch to Teradata
            success = upload_batch_to_teradata(batch_df)

            if success:
                logger.info(f"Successfully uploaded batch of {len(batch_df)} rows to Teradata")
            else:
                logger.error(f"Failed to upload batch of {len(batch_df)} rows to Teradata")

        # Repartition the DataFrame based on the desired batch size
        # This ensures each partition is approximately the batch size
        num_partitions = max(1, count // batch_size + (1 if count % batch_size > 0 else 0))
        logger.info(f"Repartitioning DataFrame into {num_partitions} partitions")

        # Process each partition
        df.repartition(num_partitions).foreachPartition(process_partition)

        logger.info(f"Successfully processed all partitions")
        return True
    except Exception as e:
        logger.error(f"Error uploading CSV to Teradata: {str(e)}")
        return False

In [ ]:
def extract_ner_entities(iterator):
    """
    Process a partition of data and apply NER extraction.
    Uses broadcast variables for models and labels.
    """
    from pyspark.taskcontext import TaskContext
    from itertools import chain

    # Get partition ID for logging
    partition_id = TaskContext.get().partitionId()
    count_in = 0
    count_out = 0
    count_errors = 0

    results = []

    logging.info(f"Partition {partition_id}: Starting processing")

    # Process each row in the partition
    for row in iterator:
        count_in += 1
        try:
            # Log the input row for debugging
            logging.debug(f"Partition {partition_id}: Processing row {count_in}: {row}")

            # Skip empty descriptions
            if not row.description or pd.isna(row.description):
                logging.warning(f"Partition {partition_id}: Empty description for company_id {row.company_id}")
                continue

            # Extract NER features from description
            ner_features = {}
            entity_sets = {}

            # Get models from broadcast variable
            model1, model2 = broadcast_models.value
            labels_dict = labels.value

            with torch.no_grad():
                # Predict entities using both models
                entities = model1.predict_entities(row.description, labels_dict, threshold=0.54)
                entities_2 = model2.predict_entities(row.description, labels_dict, threshold=0.7)

                # Check if any entities were found
                if not entities and not entities_2:
                    logging.debug(f"Partition {partition_id}: No entities found in text: {row.description[:100]}...")

                # Clean and sort entities
                entities = sorted(
                    (clean_entity(entity) for entity in chain(entities, entities_2)),
                    key=lambda k: (k['label'], -k['score'])
                )

            # Process entities and group them by label
            for entity in entities:
                label = entity["label"]
                text = entity["text"]

                # Skip empty entities
                if not text:
                    continue

                # Clean specific entity types
                if label == "brand":
                    text = text.replace('brands', '').replace('brand', '').strip()
                elif label == "industry":
                    text = text.lower().replace('industry', '').strip()
                elif label == "product":
                    text = text.lower().replace('products', '').replace('product', '').strip()

                # Add to entity sets
                if label not in entity_sets:
                    entity_sets[label] = set()
                entity_sets[label].add(text)

                # Handle "year started" separately
                if label == "year started" and "year_started" not in ner_features:
                    ner_features["year_started"] = text

            # Combine entities into a single dictionary
            fields = [
                "year_started", "product", "brand", "business_categories",
                "business_sub_categories", "industry", "business_services", "offer"
            ]
            for field in fields:
                ner_features[field] = " | ".join(entity_sets.get(field, []))

            # Post-process the row
            ner_features = post_process_row(ner_features)

            # Create result tuple
            result = (
                row.company_id,
                row.es_ecid,
                row.description,
                row.industry_primary,
                row.industries,
                row.sub_industries,
                row.top3_industries,
                ner_features.get("year_started", ""),
                ner_features.get("product", ""),
                ner_features.get("brand", ""),
                ner_features.get("industry", ""),
                ner_features.get("business_categories", ""),
                ner_features.get("business_sub_categories", ""),
                ner_features.get("business_services", ""),
                ner_features.get("offer", "")
            )

            results.append(result)
            count_out += 1

            # Log progress periodically
            if count_in % 1000 == 0:
                logging.info(f"Partition {partition_id}: Processed {count_in} rows, generated {count_out} results")

        except Exception as e:
            count_errors += 1
            if count_errors < 10:  # Limit logging to avoid overwhelming logs
                logging.error(f"Partition {partition_id}: Error processing row: {e}")
                import traceback
                logging.error(traceback.format_exc())
            continue

    # Log partition processing stats
    logging.info(f"Partition {partition_id}: Completed processing {count_in} rows, generated {count_out} results, encountered {count_errors} errors")

    # Return results as an iterator
    return iter(results)

In [ ]:


def extract_ner_features(text):
    """Extract NER features from text using broadcast models"""
    information = {}
    entity_sets = {}

    # Get models from broadcast variable
    model1, model2 = broadcast_models.value
    labels_dict = labels.value

    try:
        with torch.no_grad():
            # Predict entities using both models
            entities = model1.predict_entities(text, labels_dict, threshold=0.54)
            entities_2 = model2.predict_entities(text, labels_dict, threshold=0.7)

            # Check if any entities were found
            if not entities and not entities_2:
                logging.debug(f"No entities found in text: {text[:100]}...")

            entities = sorted(
                (clean_entity(entity) for entity in chain(entities, entities_2)),
                key=lambda k: (k['label'], -k['score'])
            )

        # Process entities and group them by label
        for entity in entities:
            label = entity["label"]
            text = entity["text"]

            # Skip empty entities
            if not text:
                continue

            # Clean specific entity types
            if label == "brand":
                text = text.replace('brands', '').replace('brand', '').strip()
            elif label == "industry":
                text = text.lower().replace('industry', '').strip()
            elif label == "product":
                text = text.lower().replace('products', '').replace('product', '').strip()

            # Add to entity sets
            if label not in entity_sets:
                entity_sets[label] = set()
            entity_sets[label].add(text)

            # Handle "year started" separately
            if label == "year started" and "year started" not in information:
                information["year_started"] = text

        # Combine entities into a single dictionary
        fields = [
            "year_started", "product", "brand", "business_categories",
            "business_sub_categories", "industry", "business_services", "offer"
        ]
        for field in fields:
            information[field] = " | ".join(entity_sets.get(field, []))

    except Exception as e:
        logging.error(f"Error in extract_ner_features: {e}")
        # Return empty information to avoid failing the entire partition
        for field in ["year_started", "product", "brand", "business_categories",
                     "business_sub_categories", "industry", "business_services", "offer"]:
            information[field] = ""

    return information

In [ ]:
def process_and_load_dataframe(spark_df, table_name, output_path="/path/to/parquet/output"):
    """
    Process the Spark DataFrame with NER extraction and save as parquet files.
    Then load parquet files to Teradata.
    """
    # Force an action to ensure the DataFrame is distributed
    row_count = spark_df.count()
    logging.info(f"Processing {row_count} records across the cluster")

    # Get cluster information to verify distribution
    sc = spark.sparkContext
    total_executors = len(sc._jsc.sc().statusTracker().getExecutorInfos()) - 1  # Subtract driver
    logging.info(f"Running on {total_executors} executors")

    # Calculate optimal partition count based on cluster size
    executor_cores = int(sc.getConf().get("spark.executor.cores", "1"))
    total_cores = total_executors * executor_cores
    num_partitions = total_cores * 3

    # Ensure we don't create too many small partitions
    records_per_partition = max(10000, row_count // num_partitions)
    num_partitions = max(total_cores, row_count // records_per_partition)

    logging.info(f"Repartitioning to {num_partitions} partitions for optimal distribution")
    spark_df = spark_df.repartition(num_partitions)

    # Initialize and broadcast models and labels
    model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
    model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")

    # Broadcast models and labels
    global broadcast_models, labels
    broadcast_models = sc.broadcast((model1, model2))

    labels_dict = {
        "headquarter": ["head office", "HQ"],
        "company name": ["company", "provider", "organization", "employer", "firm", "business"],
        "product": None,
        "brand": None,
        "industry": None,
        "business_categories": None,
        "business_sub_categories": None,
        "business_services": None,
        "offer": None,
        "year started": None
    }
    labels = sc.broadcast(labels_dict)

    # Process with mapPartitions
    processed_rdd = spark_df.rdd.mapPartitions(extract_ner_entities)

    # Force evaluation to check if RDD is empty
    sample_count = processed_rdd.take(10)
    logging.info(f"Sample processed records: {len(sample_count)}")

    if len(sample_count) == 0:
        logging.error("No records produced after processing! Debugging required.")
        # Check if input data has valid descriptions
        sample_input = spark_df.select("description").limit(5).collect()
        logging.info(f"Sample input descriptions: {sample_input}")
        raise ValueError("Processing produced empty results. Check logs for details.")

    # Convert to DataFrame with schema
    processed_df = spark.createDataFrame(processed_rdd, schema=processed_schema)

    # Force action to materialize and check count
    result_count = processed_df.count()
    logging.info(f"Processed DataFrame contains {result_count} records")

    if result_count == 0:
        logging.error("Processed DataFrame is empty! Debugging required.")
        return None

    # Cache the DataFrame to avoid recomputation
    processed_df.cache()

    # Write as partitioned parquet files
    processed_df.write \
        .mode("overwrite") \
        .option("compression", "snappy") \
        .partitionBy("industry_primary") \
        .parquet(output_path)

    logging.info(f"Successfully wrote {result_count} records to parquet files at {output_path}")

    # Define function to load partition to Teradata
    def load_partition_to_teradata(partition):
        """Load a partition of data to Teradata with tracking"""
        from pyspark.taskcontext import TaskContext
        rows = list(partition)
        partition_id = TaskContext.get().partitionId()

        if not rows:
            logging.warning(f"Partition {partition_id} has no data to upload")
            return

        logging.info(f"Partition {partition_id} uploading {len(rows)} rows to Teradata")

        try:
            with teradatasql.connect(host="hostname", user="user_id", password="password", logmech="logmech") as con:
                insert_sql = f"""
                INSERT INTO {table_name} (
                    company_id, es_ecid, description, industry_primary,
                    industries, sub_industries, top3_industries, year_started, product,
                    brand, industry, business_categories, business_sub_categories,
                    business_services, offer
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """

                # Use batch insert
                batch_size = 1000
                for i in range(0, len(rows), batch_size):
                    try:
                        batch = rows[i:i + batch_size]
                        cursor = con.cursor()
                        cursor.executemany(insert_sql, batch)
                        logging.info(f"Partition {partition_id}: Uploaded batch {i//batch_size + 1}/{(len(rows)-1)//batch_size + 1}")
                    except Exception as e:
                        logging.error(f"Partition {partition_id}: Error uploading batch: {e}")
        except Exception as e:
            logging.error(f"Partition {partition_id}: Connection error: {e}")

    # Load to Teradata in parallel
    processed_df.foreachPartition(load_partition_to_teradata)

    logging.info(f"Processing and loading of {row_count} records completed.")

    # Unpersist the cached DataFrame and broadcast variables
    processed_df.unpersist()
    broadcast_models.unpersist()
    labels.unpersist()

    return processed_df

In [ ]:
def diagnose_processing_issues(spark_df, sample_size=10):
    """
    Diagnose issues with the processing pipeline by testing each component.
    """
    logging.info("Starting diagnostic process...")

    # 1. Check input data
    sample_df = spark_df.limit(sample_size)
    sample_count = sample_df.count()
    logging.info(f"Input sample contains {sample_count} records")

    # 2. Check descriptions
    empty_descriptions = sample_df.filter(
        (F.col("description").isNull()) |
        (F.length(F.col("description")) == 0)
    ).count()
    logging.info(f"Sample contains {empty_descriptions} empty descriptions")

    # 3. Test model loading
    try:
        model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
        model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")
        logging.info("Models loaded successfully")

        # 4. Test entity extraction on a sample text
        test_text = "Apple Inc. was founded in 1976 and sells iPhones and MacBooks."
        labels_dict = {
            "product": None,
            "brand": None,
            "year started": None
        }

        entities1 = model1.predict_entities(test_text, labels_dict, threshold=0.54)
        entities2 = model2.predict_entities(test_text, labels_dict, threshold=0.7)

        logging.info(f"Model 1 found {len(entities1)} entities: {entities1}")
        logging.info(f"Model 2 found {len(entities2)} entities: {entities2}")

        # 5. Test on actual sample data
        sample_rows = sample_df.collect()
        for i, row in enumerate(sample_rows):
            if row.description:
                logging.info(f"Testing extraction on sample {i+1}: {row.description[:100]}...")

                entities1 = model1.predict_entities(row.description, labels_dict, threshold=0.54)
                entities2 = model2.predict_entities(row.description, labels_dict, threshold=0.7)

                logging.info(f"  Model 1 found {len(entities1)} entities")
                logging.info(f"  Model 2 found {len(entities2)} entities")

                # Test full extraction
                ner_features = extract_ner_features(row.description)
                logging.info(f"  Extracted features: {ner_features}")
                break

    except Exception as e:
        logging.error(f"Error during model testing: {e}")
        import traceback
        logging.error(traceback.format_exc())

    logging.info("Diagnostic process completed")

In [ ]:
# Run diagnostics before main processing
diagnose_processing_issues(spark_df)

# Then run the main processing
result_df = process_and_load_dataframe(spark_df, table_name, output_path)

V2 - Debug

In [ ]:
def test_extract_entities(iterator):
    """
    Simplified version of extract_ner_entities to isolate the issue.
    """
    from pyspark.taskcontext import TaskContext

    partition_id = TaskContext.get().partitionId()
    logging.info(f"Partition {partition_id}: Starting test extraction")

    results = []
    count = 0

    for row in iterator:
        count += 1
        logging.info(f"Partition {partition_id}: Processing row {count}")

        # Just create a simple result tuple without any processing
        result = (
            row.company_id,
            row.es_ecid,
            row.description[:100],  # Just take first 100 chars of description
            row.industry_primary,
            row.industries,
            row.sub_industries,
            row.top3_industries,
            "test_year",  # Placeholder values for the rest
            "test_product",
            "test_brand",
            "test_industry",
            "test_categories",
            "test_subcategories",
            "test_services",
            "test_offer"
        )

        results.append(result)
        logging.info(f"Partition {partition_id}: Added result {count}")

    logging.info(f"Partition {partition_id}: Completed processing {count} rows, generated {len(results)} results")

    return iter(results)

# Test with simplified function
test_rdd = spark_df.limit(5).rdd.mapPartitions(test_extract_entities)
test_results = test_rdd.collect()
logging.info(f"Simplified test results: {len(test_results)}")
logging.info(f"First result: {test_results[0] if test_results else 'None'}")

In [ ]:
# Check row structure
sample_row = spark_df.limit(1).collect()[0]
logging.info(f"Sample row structure: {sample_row}")
logging.info(f"Sample row attributes: {dir(sample_row)}")

In [ ]:
def extract_ner_entities(iterator):
    """
    Process a partition of data and apply NER extraction.
    Uses broadcast variables for models and labels.
    """
    from pyspark.taskcontext import TaskContext
    from itertools import chain
    import traceback

    # Get partition ID for logging
    partition_id = TaskContext.get().partitionId()
    logging.info(f"Partition {partition_id}: Starting processing")

    # Check if iterator is empty
    iterator_list = list(iterator)
    if not iterator_list:
        logging.warning(f"Partition {partition_id}: Empty iterator received")
        return iter([])

    logging.info(f"Partition {partition_id}: Processing {len(iterator_list)} rows")

    count_in = 0
    count_out = 0
    count_errors = 0
    results = []

    # Process each row in the partition
    for row in iterator_list:
        count_in += 1
        logging.info(f"Partition {partition_id}: Processing row {count_in}")

        try:
            # Check row structure
            logging.info(f"Row type: {type(row)}")
            logging.info(f"Row attributes: {dir(row)}")

            # Skip empty descriptions
            if not hasattr(row, 'description') or not row.description:
                logging.warning(f"Partition {partition_id}: Missing or empty description")
                continue

            logging.info(f"Description: {row.description[:100]}...")

            # Extract NER features
            try:
                # Get models from broadcast variable
                model1, model2 = broadcast_models.value
                labels_dict = labels.value

                logging.info(f"Models loaded successfully")

                # Create a simple result without any complex processing
                result = (
                    getattr(row, 'company_id', 'unknown'),
                    getattr(row, 'es_ecid', 'unknown'),
                    row.description[:100],
                    getattr(row, 'industry_primary', 'unknown'),
                    getattr(row, 'industries', 'unknown'),
                    getattr(row, 'sub_industries', 'unknown'),
                    getattr(row, 'top3_industries', 'unknown'),
                    "test_year",
                    "test_product",
                    "test_brand",
                    "test_industry",
                    "test_categories",
                    "test_subcategories",
                    "test_services",
                    "test_offer"
                )

                results.append(result)
                count_out += 1
                logging.info(f"Added result {count_out}")

            except Exception as e:
                logging.error(f"Error in NER extraction: {e}")
                logging.error(traceback.format_exc())
                continue

        except Exception as e:
            count_errors += 1
            logging.error(f"Partition {partition_id}: Error processing row: {e}")
            logging.error(traceback.format_exc())
            continue

    logging.info(f"Partition {partition_id}: Completed processing {count_in} rows, generated {count_out} results, encountered {count_errors} errors")
    logging.info(f"Results: {results[:2]}")

    return iter(results)

In [ ]:
# Test with modified function
test_rdd = spark_df.limit(5).rdd.mapPartitions(extract_ner_entities)
test_results = test_rdd.collect()
logging.info(f"Modified test results: {len(test_results)}")
logging.info(f"First result: {test_results[0] if test_results else 'None'}")

In [ ]:
# check schema compatability

# Define schema explicitly
from pyspark.sql.types import StructType, StructField, StringType

processed_schema = StructType([
    StructField("company_id", StringType(), True),
    StructField("es_ecid", StringType(), True),
    StructField("description", StringType(), True),
    StructField("industry_primary", StringType(), True),
    StructField("industries", StringType(), True),
    StructField("sub_industries", StringType(), True),
    StructField("top3_industries", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

# Test with simplified function and explicit schema
test_rdd = spark_df.limit(5).rdd.mapPartitions(test_extract_entities)
test_df = spark.createDataFrame(test_rdd, schema=processed_schema)
test_count = test_df.count()
logging.info(f"Test DataFrame count: {test_count}")

# Updated 3/7

In [ ]:
import pickle
import zlib
import time
import socket
import os
import logging
import re
from itertools import chain
from pyspark.taskcontext import TaskContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, MapType, FloatType

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='ner_processing.log',
    filemode='w'
)

# Create a logger
logger = logging.getLogger(__name__)

# Import inflect for singular noun conversion
try:
    import inflect
    p = inflect.engine()
except ImportError:
    logger.warning("Inflect package not found. Singular noun conversion will be skipped.")
    # Create a dummy p object with a method that returns False
    class DummyInflect:
        def singular_noun(self, text):
            return False
    p = DummyInflect()

# Define schemas
def get_ner_schema():
    """
    Define the schema for NER entities.
    """
    return ArrayType(
        StructType([
            StructField("text", StringType(), True),
            StructField("label", StringType(), True),
            StructField("score", FloatType(), True),
            StructField("start", StringType(), True),
            StructField("end", StringType(), True)
        ])
    )

def get_processed_schema(input_schema):
    """
    Add NER fields to the input schema.

    Args:
        input_schema: Original DataFrame schema

    Returns:
        Updated schema with NER fields
    """
    # Create a copy of the input schema
    fields = list(input_schema.fields)

    # Add the entities field if it doesn't exist
    if not any(field.name == "entities" for field in fields):
        fields.append(StructField("entities", get_ner_schema(), True))

    return StructType(fields)

def clean_entity(entity):
    """
    Clean and normalize entity text based on entity label.

    Args:
        entity: Dictionary containing entity information

    Returns:
        Cleaned entity dictionary
    """
    try:
        if entity['label'] == 'year started':
            match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
            entity['text'] = match.group(0) if match else ''
        elif entity['label'] not in ['brand']:
            entity['text'] = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
            entity['text'] = p.singular_noun(entity['text']) or entity['text']
    except Exception as e:
        logger.error(f"Error cleaning entity: {e}")
    return entity

def process_batch_with_ner(batch, model1, model2, labels):
    """
    Process a batch of records with NER models.

    Args:
        batch: List of Row objects to process
        model1: First NER model
        model2: Second NER model
        labels: Broadcast variable containing labels

    Returns:
        List of processed records with NER entities
    """
    results = []

    for row in batch:
        try:
            # Convert Row to dictionary for easier manipulation
            record = row.asDict()

            # Extract text from the record - adjust field names as needed
            text = record.get('text', '')
            if not text:
                # Skip records with no text
                results.append(row)
                continue

            # Process with model1
            entities1 = model1.extract_entities(text)

            # Process with model2
            entities2 = model2.extract_entities(text)

            # Clean and combine entities
            entities = sorted(
                (clean_entity(entity) for entity in chain(entities1, entities2)),
                key=lambda k: (k['label'], -k['score'])
            )

            # Filter by labels if needed
            if labels.value:
                entities = [e for e in entities if e['label'] in labels.value]

            # Add entities to the record
            record['entities'] = entities

            # Convert back to Row-like structure and append to results
            from pyspark.sql import Row
            results.append(Row(**record))

        except Exception as e:
            logger.error(f"Error processing record: {str(e)}")
            # Keep the original record in case of error
            results.append(row)

    return results

def extract_ner_entities(iterator):
    """
    Process each partition with batching to improve efficiency.
    Uses broadcast variables that are already in scope.
    """
    # Get the partition ID and executor information for logging
    context = TaskContext.get()
    partition_id = context.partitionId()

    # Get a unique identifier for the executor
    hostname = socket.gethostname()
    pid = str(os.getpid())
    executor_id = f"{hostname}_{pid}"

    start_time = time.time()
    logger.info(f"Starting partition {partition_id} on executor {executor_id}")

    try:
        # Log the decompression of broadcast variables - only once per partition
        logger.info(f"Partition {partition_id}: Starting model decompression")
        decompression_start = time.time()

        # Decompress models
        model1 = pickle.loads(zlib.decompress(broadcast_model1.value))
        model2 = pickle.loads(zlib.decompress(broadcast_model2.value))

        decompression_time = time.time() - decompression_start
        logger.info(f"Partition {partition_id}: Models decompressed in {decompression_time:.2f} seconds")

        # Initialize counters
        record_count = 0
        batch_count = 0
        processing_start = time.time()

        # Process in batches
        batch_size = 100  # Adjust based on your needs
        current_batch = []
        results = []

        for row in iterator:
            record_count += 1
            current_batch.append(row)

            # Process when batch is full
            if len(current_batch) >= batch_size:
                batch_count += 1
                batch_start = time.time()

                processed_batch = process_batch_with_ner(current_batch, model1, model2, labels)
                results.extend(processed_batch)

                batch_time = time.time() - batch_start
                logger.info(f"Partition {partition_id}: Processed batch {batch_count} ({len(current_batch)} records) in {batch_time:.2f} seconds")

                # Clear the batch
                current_batch = []

        # Process any remaining records in the last batch
        if current_batch:
            batch_count += 1
            batch_start = time.time()

            processed_batch = process_batch_with_ner(current_batch, model1, model2, labels)
            results.extend(processed_batch)

            batch_time = time.time() - batch_start
            logger.info(f"Partition {partition_id}: Processed final batch {batch_count} ({len(current_batch)} records) in {batch_time:.2f} seconds")

        # Log completion
        total_time = time.time() - start_time
        logger.info(f"Finished partition {partition_id} on {executor_id}. "
                   f"Processed {record_count} records in {total_time:.2f} seconds")

        return iter(results)

    except Exception as e:
        logger.error(f"Major error in partition {partition_id} on {executor_id}: {str(e)}")
        import traceback
        logger.error(f"Traceback: {traceback.format_exc()}")
        # Re-raise to ensure Spark knows about the failure
        raise

def process_and_load_dataframe(spark_df, output_path=None, teradata_table=None):
    """
    Process a Spark DataFrame with NER extraction and load to output destination.

    Args:
        spark_df: Input Spark DataFrame
        output_path: Optional path to write parquet files
        teradata_table: Optional Teradata table name to write results

    Returns:
        Processed DataFrame
    """
    start_time = time.time()
    logger.info("Starting NER processing")

    # Get Spark context
    spark = SparkSession.builder.getOrCreate()
    sc = spark.sparkContext

    try:
        # Log initial DataFrame information
        partition_count = spark_df.rdd.getNumPartitions()
        record_count = spark_df.count()
        logger.info(f"Input DataFrame has {record_count} records in {partition_count} partitions")

        # Repartition if needed for better distribution
        target_partitions = min(max(120, record_count // 1000), 400)  # Between 120 and 400 partitions
        if partition_count != target_partitions:
            logger.info(f"Repartitioning from {partition_count} to {target_partitions} partitions")
            spark_df = spark_df.repartition(target_partitions)

        # Get the processed schema
        input_schema = spark_df.schema
        processed_schema = get_processed_schema(input_schema)
        logger.info(f"Using processed schema: {processed_schema}")

        # Process the DataFrame
        logger.info("Processing DataFrame with NER extraction")
        processed_rdd = spark_df.rdd.mapPartitions(extract_ner_entities)

        # Convert back to DataFrame with the correct schema
        processed_df = spark.createDataFrame(processed_rdd, processed_schema)

        # Force an action to ensure processing happens
        processed_count = processed_df.count()
        logger.info(f"Processed {processed_count} records")

        # Write output if path is provided
        if output_path:
            logger.info(f"Writing results to {output_path}")
            # Coalesce to reduce number of files for writing
            write_partitions = min(20, target_partitions // 5)  # Reduce file count for writing
            processed_df.coalesce(write_partitions).write \
                .option("compression", "snappy") \
                .mode("overwrite") \
                .parquet(output_path)
            logger.info(f"Results written to {output_path}")

        # Write to Teradata if table name is provided
        if teradata_table:
            logger.info(f"Writing results to Teradata table {teradata_table}")
            # Use Teradata JDBC connection to write data
            processed_df.write \
                .format("jdbc") \
                .option("url", "jdbc:teradata://your_teradata_server") \
                .option("dbtable", teradata_table) \
                .option("user", "your_username") \
                .option("password", "your_password") \
                .mode("overwrite") \
                .save()
            logger.info(f"Results written to Teradata table {teradata_table}")

        # Log completion
        total_time = time.time() - start_time
        logger.info(f"NER processing completed in {total_time:.2f} seconds")

        return processed_df

    except Exception as e:
        logger.error(f"Error in process_and_load_dataframe: {str(e)}")
        import traceback
        logger.error(f"Traceback: {traceback.format_exc()}")
        raise

In [ ]:
## Simplified batching

In [ ]:
import torch
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import logging
import re
from iterabilities import chain
import inflect
import teradatasql
import pandas as pd
import pickle
import zlib
from typing import List, Dict, Any, Iterator, Tuple

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Spark session
spark = SparkSession.builder \
    .appName("NER Processing with mapPartitions") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

# Define schema for the processed DataFrame
processed_schema = StructType([
    StructField("company_id", StringType(), True),
    StructField("es_ecid", StringType(), True),
    StructField("description", StringType(), True),
    StructField("industry_primary", StringType(), True),
    StructField("industries", StringType(), True),
    StructField("sub_industries", StringType(), True),
    StructField("top3_industries", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

# Pre-processing: Clean entity function
def clean_entity(entity):
    """Clean and process entity text."""
    try:
        if entity['label'] == 'year started':
            match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
            entity['text'] = match.group(0) if match else ''
        elif entity['label'] not in ['brand']:
            entity['text'] = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
            entity['text'] = inflect.engine().singular_noun(entity['text']) or entity['text']
    except Exception as e:
        logging.error(f"Error cleaning entity: {e}")
    return entity

# Post-processing: Add missing fields and clean up
def post_process_row(row):
    """Ensure all required fields are present and clean up the row."""
    required_fields = [
        "year_started", "product", "brand", "industry", "business_categories",
        "business_sub_categories", "business_services", "offer"
    ]
    for field in required_fields:
        if field not in row or row[field] is None:
            row[field] = ""
    return row

# Compress and broadcast models
# Load models
model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")

# Compress models
compressed_model1 = zlib.compress(pickle.dumps(model1))
compressed_model2 = zlib.compress(pickle.dumps(model2))

# Broadcast compressed models
broadcast_model1 = sc.broadcast(compressed_model1)
broadcast_model2 = sc.broadcast(compressed_model2)

# Define labels dictionary and broadcast it
labels_dict = {
    "headquarter": ["head office", "HQ"],
    "company name": ["company", "provider", "organization", "employer", "firm", "business"],
    "product": None,
    "brand": None
}
broadcast_labels = sc.broadcast(labels_dict)

# Batch processing function for NER extraction
def extract_ner_features(texts: List[str], model1, model2, labels_dict, batch_size=8):
    """Process texts in batches for more efficient NER extraction."""
    all_entities = []

    # Process in batches
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch_entities = []

        with torch.no_grad():
            for text in batch:
                entities1 = model1.predict_entities(text, labels_dict, threshold=0.54)
                entities2 = model2.predict_entities(text, labels_dict, threshold=0.7)
                combined = sorted(
                    (clean_entity(entity) for entity in chain(entities1, entities2)),
                    key=lambda k: (k['label'], -k['score'])
                )
                batch_entities.append(combined)

        all_entities.extend(batch_entities)

    return all_entities

# Partition processing function
def extract_ner_entities(iterator):
    """
    Process a partition of data and apply NER extraction.
    """
    # Decompress models once per partition
    model1 = pickle.loads(zlib.decompress(broadcast_model1.value))
    model2 = pickle.loads(zlib.decompress(broadcast_model2.value))
    labels_dict = broadcast_labels.value

    # Collect rows and descriptions for batch processing
    partition_rows = []
    descriptions = []

    for row in iterator:
        partition_rows.append(row)
        descriptions.append(row.description)

    # Process descriptions in batches
    all_entities = extract_ner_features(descriptions, model1, model2, labels_dict)

    # Process results for each row
    for idx, (row, entities) in enumerate(zip(partition_rows, all_entities)):
        try:
            # Combine entities into a single dictionary
            ner_features = {}
            entity_sets = {}
            for entity in entities:
                label = entity["label"]
                text = entity["text"]

                if label not in entity_sets:
                    entity_sets[label] = set()
                entity_sets[label].add(text)

                if label == "year started" and "year_started" not in ner_features:
                    ner_features["year_started"] = text

            # Fill in all fields
            fields = [
                "year_started", "product", "brand", "industry", "business_categories",
                "business_sub_categories", "business_services", "offer"
            ]
            for field in fields:
                ner_features[field] = " | ".join(entity_sets.get(field, []))

            # Post-process the row
            ner_features = post_process_row(ner_features)

            # Yield the processed row
            yield (
                row.company_id,
                row.es_ecid,
                row.description,
                row.industry_primary,
                row.industries,
                row.sub_industries,
                row.top3_industries,
                ner_features["year_started"],
                ner_features["product"],
                ner_features["brand"],
                ner_features["industry"],
                ner_features["business_categories"],
                ner_features["business_sub_categories"],
                ner_features["business_services"],
                ner_features["offer"]
            )
        except Exception as e:
            logging.error(f"Error processing row {idx}: {e}")
            continue

# Upload to Teradata
def upload_to_teradata(df, table_name, batch_size_db):
    """Upload data to Teradata using teradatasql."""
    with teradatasql.connect(host="hostname", user="user_id", password="password", logmech="logmech") as con:
        insert_sql = f"""
        INSERT INTO {table_name} (
            company_id, es_ecid, description, industry_primary,
            industries, sub_industries, top3_industries, year_started, product,
            brand, industry, business_categories, business_sub_categories,
            business_services, offer
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        data = [tuple(x) for x in df.values]

        for i in range(0, len(data), batch_size_db):
            try:
                batch = data[i:i + batch_size_db]
                cursor = con.cursor()
                cursor.executemany(insert_sql, batch)
                logging.info(f"Batch {i // batch_size_db + 1} uploaded successfully.")
            except Exception as e:
                logging.error(f"Error uploading batch {i // batch_size_db + 1}: {e}")
                logging.error(f"Offending batch: {batch}")
                continue

# Main processing function
def process_and_load_dataframe(spark_df, table_name, batch_size_db=100):
    """
    Process the Spark DataFrame with NER extraction using mapPartitions and upload results to Teradata.
    """
    # Repartition for better distribution
    spark_df = spark_df.repartition(64)  # Adjust based on dataset size and cluster configuration
    logging.info(f"Repartitioned DataFrame into {spark_df.rdd.getNumPartitions()} partitions.")

    # Convert DataFrame to RDD and apply mapPartitions
    processed_rdd = spark_df.rdd.mapPartitions(extract_ner_entities)

    # Convert the RDD back to a DataFrame
    processed_df = spark.createDataFrame(processed_rdd, schema=processed_schema)

    # Log metrics after processing
    logging.info(f"Processed DataFrame contains {processed_df.count()} records after NER extraction.")
    logging.info(f"Number of partitions after processing: {processed_df.rdd.getNumPartitions()}")

    # Convert Spark DataFrame to Pandas for batch upload
    pandas_df = processed_df.toPandas()

    # Upload each batch to Teradata
    for i in range(0, len(pandas_df), batch_size_db):
        batch = pandas_df.iloc[i:i + batch_size_db]
        logging.info(f"Uploading batch {i // batch_size_db + 1} with {len(batch)} records to Teradata.")
        upload_to_teradata(batch, table_name, batch_size_db)

Update 3_8

In [ ]:
# Import necessary libraries
import os
import re
import gc
import time
import zlib
import pickle
import socket
import logging
from itertools import chain
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.taskcontext import TaskContext
from gliner import GLiNER

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='ner_processing.log',
    filemode='w'
)
logger = logging.getLogger("NER_Processing")

# Initialize Spark session
spark = SparkSession.builder \
    .appName("NER Processing") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.sql.files.maxPartitionBytes", "134217728") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "200") \
    .getOrCreate()

sc = spark.sparkContext

# Define schemas and labels
labels = {
    "year started": None,
    "product": None,
    "brand": None,
    "business categories": None,
    "business sub categories": None,
    "industry": None,
    "business services": None,
    "offer": None,
}

ner_schema = StructType([
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

processed_schema = StructType([
    StructField("company_id", StringType(), True),
    StructField("es_ecid", StringType(), True),
    StructField("description", StringType(), True),
    StructField("industry_primary", StringType(), True),
    StructField("industries", StringType(), True),
    StructField("sub_industries", StringType(), True),
    StructField("top3_industries", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

# Load and compress models
logger.info("Loading NER models")
model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")

logger.info("Compressing models")
compressed_model1 = zlib.compress(pickle.dumps(model1))
compressed_model2 = zlib.compress(pickle.dumps(model2))

# Free up memory
del model1
del model2
gc.collect()

# Broadcast variables
broadcast_model1 = sc.broadcast(compressed_model1)
broadcast_model2 = sc.broadcast(compressed_model2)
broadcast_labels = sc.broadcast(list(labels.keys()))

def clean_entity(entity):
    """Clean and normalize entity text."""
    try:
        if entity['label'] == 'year started':
            match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
            return {'label': entity['label'], 'text': match.group(0) if match else '', 'score': entity['score']}
        elif entity['label'] not in ['brand']:
            cleaned_text = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
            return {'label': entity['label'], 'text': cleaned_text, 'score': entity['score']}
    except Exception as e:
        logger.error(f"Error cleaning entity: {e}")
    return entity

def map_entity_to_field(entity_label):
    """Map entity label to schema field name."""
    mapping = {
        "year started": "year_started",
        "product": "product",
        "brand": "brand",
        "business categories": "business_categories",
        "business sub categories": "business_sub_categories",
        "industry": "industry",
        "business services": "business_services",
        "offer": "offer"
    }
    return mapping.get(entity_label, entity_label)

def extract_ner_entities(iterator):
    """Process each partition with NER extraction."""
    # Get partition ID for logging
    partition_id = TaskContext.get().partitionId()

    try:
        # Decompress models once per partition
        model1 = pickle.loads(zlib.decompress(broadcast_model1.value))
        model2 = pickle.loads(zlib.decompress(broadcast_model2.value))
        labels_list = broadcast_labels.value

        batch_size = 100
        current_batch = []

        for row in iterator:
            current_batch.append(row)

            if len(current_batch) >= batch_size:
                # Process the batch
                processed_records = []

                for record in current_batch:
                    try:
                        # Initialize a new record with all fields from processed_schema
                        new_record = {field.name: None for field in processed_schema.fields}

                        # Copy existing fields from input record
                        input_dict = record.asDict()
                        for key, value in input_dict.items():
                            if key in new_record:
                                new_record[key] = value

                        # Process text if available
                        text = input_dict.get('description', '')
                        if text:
                            # Extract entities from both models
                            entities1 = model1.extract_entities(text)
                            entities2 = model2.extract_entities(text)

                            # Combine and clean entities
                            all_entities = []
                            for entity in chain(entities1, entities2):
                                cleaned = clean_entity(entity)
                                if cleaned['label'] in labels_list:
                                    all_entities.append(cleaned)

                            # Group by label and take highest scoring
                            entity_dict = {}
                            for entity in sorted(all_entities, key=lambda x: (-x['score'], x['label'])):
                                if entity['label'] not in entity_dict:
                                    entity_dict[entity['label']] = entity

                            # Map entities to fields
                            for entity in entity_dict.values():
                                field_name = map_entity_to_field(entity['label'])
                                new_record[field_name] = entity['text']

                        processed_records.append(Row(**new_record))

                    except Exception as e:
                        logger.error(f"Error processing record in partition {partition_id}: {str(e)}")
                        # Add record with all fields (initialized to None)
                        processed_records.append(Row(**new_record))

                for record in processed_records:
                    yield record

                current_batch = []

        # Process remaining records
        if current_batch:
            for record in current_batch:
                try:
                    new_record = {field.name: None for field in processed_schema.fields}
                    input_dict = record.asDict()

                    for key, value in input_dict.items():
                        if key in new_record:
                            new_record[key] = value

                    text = input_dict.get('description', '')
                    if text:
                        entities1 = model1.extract_entities(text)
                        entities2 = model2.extract_entities(text)

                        all_entities = []
                        for entity in chain(entities1, entities2):
                            cleaned = clean_entity(entity)
                            if cleaned['label'] in labels_list:
                                all_entities.append(cleaned)

                        entity_dict = {}
                        for entity in sorted(all_entities, key=lambda x: (-x['score'], x['label'])):
                            if entity['label'] not in entity_dict:
                                entity_dict[entity['label']] = entity

                        for entity in entity_dict.values():
                            field_name = map_entity_to_field(entity['label'])
                            new_record[field_name] = entity['text']

                    yield Row(**new_record)

                except Exception as e:
                    logger.error(f"Error processing record in partition {partition_id}: {str(e)}")
                    yield Row(**new_record)

    except Exception as e:
        logger.error(f"Major error in partition {partition_id}: {str(e)}")
        raise

# Read input data
input_path = "gs://your-bucket/input_path"  # Replace with your actual input path
input_df = spark.read.parquet(input_path)

# Log initial DataFrame information
logger.info(f"Input DataFrame schema: {input_df.schema}")

# Process the DataFrame
processed_rdd = input_df.rdd.mapPartitions(extract_ner_entities)
processed_df = spark.createDataFrame(processed_rdd, processed_schema)

# Log processed DataFrame information
logger.info(f"Processed DataFrame schema: {processed_df.schema}")

# Write output
output_path = "gs://your-bucket/output_path"  # Replace with your actual output path
processed_df.coalesce(20).write \
    .option("compression", "snappy") \
    .option("maxRecordsPerFile", 50000) \
    .mode("overwrite") \
    .parquet(output_path)

logger.info(f"Results written to {output_path}")

In [ ]:
##   3_8 Updated code


# Updated labels with aliases
labels = {
    "year started": ["year started", "incorporated year", "founded year"],
    "product": ["product", "products", "offered"],
    "brand": ["brand", "logo", "trade mark"],
    "business categories": ["business categories"],
    "business sub categories": ["business sub categories"],
    "industry": ["industry"],
    "business services": ["business services"],
    "offer": ["offer"]
}

# Broadcast the updated labels
broadcast_labels = sc.broadcast(labels)

def clean_entity(entity):
    """Clean and normalize entity text."""
    try:
        if entity['label'] in labels["year started"]:
            # Extract valid year
            match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
            return {'label': "year started", 'text': match.group(0) if match else '', 'score': entity['score']}
        elif entity['label'] not in labels["brand"]:
            # Remove non-alphanumeric characters for other labels
            cleaned_text = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
            return {'label': entity['label'], 'text': cleaned_text, 'score': entity['score']}
        else:
            return entity
    except Exception as e:
        logger.error(f"Error cleaning entity: {e}")
        return entity

def map_entity_to_field(entity_label):
    """Map entity label or alias to schema field name."""
    mapping = {
        "year started": "year_started",
        "product": "product",
        "brand": "brand",
        "business categories": "business_categories",
        "business sub categories": "business_sub_categories",
        "industry": "industry",
        "business services": "business_services",
        "offer": "offer"
    }
    # Find the correct field for the label or its alias
    for key, aliases in labels.items():
        if entity_label in aliases:
            return mapping.get(key, key)
    return entity_label  # Default to the original label if no match is found

def extract_ner_entities(iterator):
    """Process each partition with NER extraction using batching."""
    from pyspark.taskcontext import TaskContext
    from itertools import chain
    import traceback

    partition_id = TaskContext.get().partitionId()
    logging.info(f"Partition {partition_id}: Starting processing")

    # Initialize counters for logging
    count_in = 0
    count_out = 0
    count_errors = 0

    # Batch size for processing
    batch_size = 100
    current_batch = []

    try:
        # Decompress models and labels once per partition
        model1 = pickle.loads(zlib.decompress(broadcast_model1.value))
        model2 = pickle.loads(zlib.decompress(broadcast_model2.value))
        labels_list = broadcast_labels.value

        for row in iterator:
            count_in += 1
            current_batch.append(row)

            # Process the batch when it reaches the batch size
            if len(current_batch) >= batch_size:
                batch_results = process_batch(
                    current_batch, model1, model2, labels_list, partition_id
                )
                for result in batch_results:
                    count_out += 1
                    yield result
                current_batch = []

        # Process any remaining rows in the final batch
        if current_batch:
            batch_results = process_batch(
                current_batch, model1, model2, labels_list, partition_id
            )
            for result in batch_results:
                count_out += 1
                yield result

    except Exception as e:
        logging.error(f"Partition {partition_id}: Major error: {str(e)}")
        logging.error(traceback.format_exc())
        raise

    logging.info(
        f"Partition {partition_id}: Completed processing {count_in} rows, "
        f"generated {count_out} results, encountered {count_errors} errors"
    )


def process_batch(batch, model1, model2, labels_list, partition_id):
    """Process a batch of rows."""
    results = []

    for row in batch:
        try:
            # Convert row to dictionary for easier access
            row_dict = row.asDict()

            # Skip rows with missing or empty descriptions
            if 'description' not in row_dict or not row_dict['description']:
                logging.warning(f"Partition {partition_id}: Missing or empty description")
                continue

            # Initialize feature holders
            ner_features = {}
            entity_sets = {}

            # Extract entities from both models
            with torch.no_grad():
                entities1 = model1.predict_entities(row_dict['description'], labels_list, threshold=0.54) or []
                entities2 = model2.predict_entities(row_dict['description'], labels_list, threshold=0.7) or []

            # Combine and clean entities
            entities = sorted(
                (clean_entity(entity) for entity in chain(entities1, entities2)),
                key=lambda k: (k['label'], -k['score'])
            )

            # Group entities by label and deduplicate
            for entity in entities:
                label = entity["label"]
                text = entity["text"]
                if not text:
                    continue

                # Clean text based on label
                if label == "brand":
                    text = text.replace('brands', '').replace('brand', '').strip()
                elif label == "industry":
                    text = text.lower().replace('industry', '').strip()
                elif label == "product":
                    text = text.lower().replace('products', '').replace('product', '').strip()

                if label not in entity_sets:
                    entity_sets[label] = set()
                entity_sets[label].add(text)

                if label == "year started" and "year_started" not in ner_features:
                    ner_features["year_started"] = text

            # Concatenate values for each field
            fields = [
                "year_started", "product", "brand", "business_categories",
                "business_sub_categories", "industry", "business_services", "offer"
            ]
            for field in fields:
                ner_features[field] = " | ".join(entity_sets.get(field, []))

            # Ensure all required fields are present
            for field in fields:
                if field not in ner_features:
                    ner_features[field] = ""

            # Build the output Row object
            result = Row(
                company_id=row_dict.get('company_id', ''),
                es_ecid=row_dict.get('es_ecid', ''),
                description=row_dict.get('description', ''),
                industry_primary=row_dict.get('industry_primary', ''),
                industries=row_dict.get('industries', ''),
                sub_industries=row_dict.get('sub_industries', ''),
                top3_industries=row_dict.get('top3_industries', ''),
                year_started=ner_features.get("year_started", ""),
                product=ner_features.get("product", ""),
                brand=ner_features.get("brand", ""),
                industry=ner_features.get("industry", ""),
                business_categories=ner_features.get("business_categories", ""),
                business_sub_categories=ner_features.get("business_sub_categories", ""),
                business_services=ner_features.get("business_services", ""),
                offer=ner_features.get("offer", "")
            )

            results.append(result)

        except Exception as e:
            logging.error(f"Partition {partition_id}: Error processing row: {str(e)}")
            logging.error(traceback.format_exc())
            continue

    return results

In [ ]:
def read_parquet_and_upload_to_teradata(gs_bucket_path, teradata_table, batch_size=10000):
    """
    Read parquet files from GCS bucket and upload to Teradata.

    Args:
        gs_bucket_path: GCS path containing parquet files
        teradata_table: Teradata table name to write data to
        batch_size: Number of records to upload in each batch

    Returns:
        None
    """
    import teradatasql
    import pandas as pd

    start_time = time.time()
    logger.info(f"Starting to read parquet files from {gs_bucket_path}")

    try:
        # Read parquet files from GCS bucket
        parquet_df = spark.read.parquet(gs_bucket_path)

        # Log DataFrame information
        record_count = parquet_df.count()
        logger.info(f"Read {record_count} records from parquet files")

        # Teradata connection parameters
        td_host = "your_teradata_host"  # Replace with your Teradata host
        td_user = "your_username"       # Replace with your username
        td_password = "your_password"   # Replace with your password

        # Establish Teradata connection
        conn_str = f"host={td_host},user={td_user},password={td_password}"

        # Get column names for the INSERT statement
        columns = parquet_df.columns
        column_str = ", ".join(columns)
        placeholders = ", ".join(["?"] * len(columns))

        # Prepare INSERT statement
        insert_stmt = f"INSERT INTO {teradata_table} ({column_str}) VALUES ({placeholders})"

        # Calculate optimal number of partitions for processing
        # Use fewer partitions for Teradata upload to reduce connection overhead
        upload_partitions = min(20, record_count // batch_size + 1)
        logger.info(f"Using {upload_partitions} partitions for Teradata upload")

        # Repartition if needed
        if parquet_df.rdd.getNumPartitions() > upload_partitions:
            parquet_df = parquet_df.coalesce(upload_partitions)

        # Define a function to process each partition
        def upload_partition(iterator):
            # Connect to Teradata
            with teradatasql.connect(conn_str) as conn:
                cursor = conn.cursor()

                # Process records in batches
                batch = []
                count = 0
                total = 0

                for row in iterator:
                    # Convert Row to tuple for insertion
                    values = tuple(getattr(row, col_name, None) for col_name in columns)
                    batch.append(values)
                    count += 1

                    # Execute when batch is full
                    if count >= batch_size:
                        try:
                            cursor.executemany(insert_stmt, batch)
                            conn.commit()
                            total += count
                            logger.info(f"Uploaded {total} records to Teradata")
                        except Exception as e:
                            logger.error(f"Error uploading batch to Teradata: {str(e)}")
                            conn.rollback()

                        # Clear the batch
                        batch = []
                        count = 0

                # Upload any remaining records
                if batch:
                    try:
                        cursor.executemany(insert_stmt, batch)
                        conn.commit()
                        total += count
                        logger.info(f"Uploaded final batch of {count} records to Teradata")
                    except Exception as e:
                        logger.error(f"Error uploading final batch to Teradata: {str(e)}")
                        conn.rollback()

                return [total]

        # Execute the upload function on each partition
        upload_counts = parquet_df.rdd.mapPartitions(upload_partition).collect()

        # Calculate total uploaded records
        total_uploaded = sum(upload_counts)
        logger.info(f"Total records uploaded to Teradata: {total_uploaded}")

        # Log completion
        total_time = time.time() - start_time
        logger.info(f"Teradata upload completed in {total_time:.2f} seconds")

    except Exception as e:
        logger.error(f"Error in read_parquet_and_upload_to_teradata: {str(e)}")
        import traceback
        logger.error(f"Traceback: {traceback.format_exc()}")
        raise

# Example usage
# read_parquet_and_upload_to_teradata(
#     gs_bucket_path="gs://your-bucket/output_path",
#     teradata_table="your_database.your_table",
#     batch_size=10000
# )

In [ ]:
# Read processed parquet files and upload to Teradata
def upload_processed_data_to_teradata(gs_bucket_path):
    """
    Read all parquet files recursively from GCS bucket and upload to Teradata
    using distributed processing
    """
    logger.info(f"Starting to read parquet files from: {gs_bucket_path}")

    try:
        # Read all parquet files recursively from the bucket
        df = spark.read \
            .option("recursiveFileLookup", "true") \
            .option("pathGlobFilter", "*.parquet") \
            .schema(processed_schema) \
            .parquet(gs_bucket_path)

        logger.info(f"Total records read: {df.count()}")
        logger.info(f"Number of partitions: {df.rdd.getNumPartitions()}")

        # Repartition if needed for better parallelism
        num_partitions = 200  # Adjust based on your cluster size
        df = df.repartition(num_partitions)

        # Convert DataFrame to list of dictionaries for batch processing
        rows = df.collect()
        total_rows = len(rows)
        logger.info(f"Processing {total_rows} rows for Teradata upload")

        # Process in batches
        batch_size = 1000
        for i in range(0, total_rows, batch_size):
            batch = rows[i:i + batch_size]

            # Convert batch to list of dictionaries
            records = [row.asDict() for row in batch]

            try:
                # Use existing upload_to_teradata function
                upload_to_teradata(records)
                logger.info(f"Successfully uploaded batch {i//batch_size + 1} to Teradata")
            except Exception as e:
                logger.error(f"Error uploading batch {i//batch_size + 1} to Teradata: {str(e)}")
                continue

        logger.info("Completed uploading all data to Teradata")

    except Exception as e:
        logger.error(f"Error processing parquet files: {str(e)}")
        raise

# Example usage:
gs_bucket_path = "gs://your-bucket/output_path/*"  # Path including all subfolders
upload_processed_data_to_teradata(gs_bucket_path)

# uplaod to Teradata

In [ ]:
from pyspark.sql.functions import col, lit
import teradatasql
import logging
from pyspark.taskcontext import TaskContext

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Connection parameters - Consider using a config file or secrets manager
conn_params = {
    "host": "hostname",
    "user": "user_id",
    "password": "password",
    "logmech": "logmech"
}

# Define target columns
columns = [
    "company_id", "es_ecid", "description", "industry_primary",
    "industries", "sub_industries", "top3_industries", "year_started", "product",
    "brand", "industry", "business_categories", "business_sub_categories",
    "business_services", "offer"
]

# Prepare DataFrame for upload
df_upload = df.select(*columns)

# Calculate optimal partitions based on data volume and available nodes
record_count = df_upload.count()
records_per_partition = 100000  # Adjust based on your data characteristics
optimal_partitions = min(100, max(20, record_count // records_per_partition))

# Repartition if needed
df_upload = df_upload.repartition(optimal_partitions)

# Prepare SQL statement
insert_sql = f"""
INSERT INTO {table_name} (
    {', '.join(columns)}
) VALUES ({', '.join(['?' for _ in columns])})
"""

# Broadcast the connection parameters and SQL to all executors
broadcast_conn = sc.broadcast(conn_params)
broadcast_sql = sc.broadcast(insert_sql)

def process_partition(iterator):
    """Process each partition and upload to Teradata"""
    partition_id = TaskContext.get().partitionId()
    batch_size = 5000  # Adjust based on Teradata performance
    current_batch = []
    results = []

    # Get connection parameters from broadcast
    conn_params = broadcast_conn.value
    insert_sql = broadcast_sql.value

    # Connect once per partition
    with teradatasql.connect(**conn_params) as conn:
        cursor = conn.cursor()

        for row in iterator:
            # Convert Row to tuple in the correct column order
            record = tuple(row[col] for col in columns)
            current_batch.append(record)

            if len(current_batch) >= batch_size:
                try:
                    cursor.executemany(insert_sql, current_batch)
                    conn.commit()
                    results.append({
                        "status": "success",
                        "partition": partition_id,
                        "records": len(current_batch)
                    })
                except Exception as e:
                    conn.rollback()
                    results.append({
                        "status": "error",
                        "partition": partition_id,
                        "records": len(current_batch),
                        "error": str(e)
                    })
                current_batch = []

        # Process remaining records
        if current_batch:
            try:
                cursor.executemany(insert_sql, current_batch)
                conn.commit()
                results.append({
                    "status": "success",
                    "partition": partition_id,
                    "records": len(current_batch)
                })
            except Exception as e:
                conn.rollback()
                results.append({
                    "status": "error",
                    "partition": partition_id,
                    "records": len(current_batch),
                    "error": str(e)
                })

    return iter(results)

# Execute upload across partitions
upload_results = df_upload.rdd.mapPartitions(process_partition).collect()

# Summarize results
success_count = sum(1 for r in upload_results if r["status"] == "success")
error_count = sum(1 for r in upload_results if r["status"] == "error")
success_records = sum(r["records"] for r in upload_results if r["status"] == "success")
error_records = sum(r["records"] for r in upload_results if r["status"] == "error")

logging.info(f"""
Upload Summary:
- Successful partitions: {success_count}
- Failed partitions: {error_count}
- Successfully uploaded records: {success_records}
- Failed records: {error_records}
""")

Update 3_8 V1 - Debugging

In [ ]:
# Import necessary libraries
import zlib
import pickle
from itertools import chain
from pyspark.sql import Row
from gliner import GLiNER

# Sample company description
sample_description = """
Apple Inc. was founded in 1976 by Steve Jobs. The company is a leading technology brand
that designs and manufactures consumer electronics products like iPhone, iPad, and MacBook.
They operate in the consumer electronics industry, offering software services and hardware products.
Apple Store provides repair services and product support to customers with special offers on new launches.
"""

# Create a sample input record
sample_record = Row(
    company_id="12345",
    es_ecid="ES123",
    description=sample_description,
    industry_primary="Technology",
    industries="Consumer Electronics",
    sub_industries="Mobile Devices",
    top3_industries="Technology|Electronics|Software"
)

# Load models directly for testing
print("Loading models...")
model1 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")
model2 = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")

# Define labels
labels = [
    "year started", "product", "brand", "business categories",
    "business sub categories", "industry", "business services", "offer"
]

print("\n1. Extracting entities from model1...")
entities1 = model1.extract_entities(sample_description)
print("Model 1 entities:")
for entity in entities1:
    print(f"Label: {entity['label']}, Text: {entity['text']}, Score: {entity['score']}")

print("\n2. Extracting entities from model2...")
entities2 = model2.extract_entities(sample_description)
print("Model 2 entities:")
for entity in entities2:
    print(f"Label: {entity['label']}, Text: {entity['text']}, Score: {entity['score']}")

print("\n3. Cleaning and filtering entities...")
all_entities = []
for entity in chain(entities1, entities2):
    # Clean entity
    if entity['label'] == 'year started':
        import re
        match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
        cleaned = {
            'label': entity['label'],
            'text': match.group(0) if match else '',
            'score': entity['score']
        }
    elif entity['label'] not in ['brand']:
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
        cleaned = {
            'label': entity['label'],
            'text': cleaned_text,
            'score': entity['score']
        }
    else:
        cleaned = entity

    if cleaned['label'] in labels:
        all_entities.append(cleaned)

print("Cleaned and filtered entities:")
for entity in all_entities:
    print(f"Label: {entity['label']}, Text: {entity['text']}, Score: {entity['score']}")

print("\n4. Grouping by label and taking highest scoring...")
entity_dict = {}
for entity in sorted(all_entities, key=lambda x: (-x['score'], x['label'])):
    if entity['label'] not in entity_dict:
        entity_dict[entity['label']] = entity

print("Final grouped entities:")
for label, entity in entity_dict.items():
    print(f"Label: {label}, Text: {entity['text']}, Score: {entity['score']}")

print("\n5. Creating final record...")
# Initialize new record with schema fields
new_record = {
    "company_id": sample_record.company_id,
    "es_ecid": sample_record.es_ecid,
    "description": sample_record.description,
    "industry_primary": sample_record.industry_primary,
    "industries": sample_record.industries,
    "sub_industries": sample_record.sub_industries,
    "top3_industries": sample_record.top3_industries,
    "year_started": None,
    "product": None,
    "brand": None,
    "industry": None,
    "business_categories": None,
    "business_sub_categories": None,
    "business_services": None,
    "offer": None
}

# Map entities to fields
for entity in entity_dict.values():
    field_name = entity['label'].replace(" ", "_")
    new_record[field_name] = entity['text']

print("Final processed record:")
for key, value in new_record.items():
    print(f"{key}: {value}")

In [ ]:
def extract_ner_entities(iterator):
    """Process each partition with NER extraction."""
    # Get partition ID for logging
    partition_id = TaskContext.get().partitionId()
    logger.info(f"Starting partition {partition_id}")

    try:
        # Decompress models once per partition
        model1 = pickle.loads(zlib.decompress(broadcast_model1.value))
        model2 = pickle.loads(zlib.decompress(broadcast_model2.value))
        labels_dict = broadcast_labels.value

        # Get all possible labels including aliases
        all_labels = list(chain(*labels_dict.values()))

        batch_size = 100
        current_batch = []

        for row in iterator:
            current_batch.append(row)

            if len(current_batch) >= batch_size:
                # Process the batch
                processed_records = []

                for record in current_batch:
                    try:
                        # Initialize a new record with all fields from processed_schema
                        new_record = {field.name: None for field in processed_schema.fields}

                        # Copy existing fields from input record
                        input_dict = record.asDict()
                        for key, value in input_dict.items():
                            if key in new_record:
                                new_record[key] = value

                        # Process text if available
                        text = input_dict.get('description', '')
                        if text:
                            # Extract entities from both models using predict_entities
                            entities1 = model1.predict_entities(text, all_labels, threshold=0.54)
                            entities2 = model2.predict_entities(text, all_labels, threshold=0.7)

                            logger.debug(f"Entities from model1: {entities1}")
                            logger.debug(f"Entities from model2: {entities2}")

                            # Combine and clean entities
                            all_entities = []
                            for entity in chain(entities1, entities2):
                                cleaned = clean_entity(entity)
                                # Check if the label or its aliases are valid
                                for main_label, aliases in labels_dict.items():
                                    if cleaned['label'] in aliases:
                                        cleaned['label'] = main_label
                                        all_entities.append(cleaned)
                                        break

                            logger.debug(f"Combined and cleaned entities: {all_entities}")

                            # Group by label and take highest scoring
                            entity_dict = {}
                            for entity in sorted(all_entities, key=lambda x: (-x['score'], x['label'])):
                                if entity['label'] not in entity_dict:
                                    entity_dict[entity['label']] = entity

                            logger.debug(f"Final entity dictionary: {entity_dict}")

                            # Map entities to fields
                            for entity in entity_dict.values():
                                field_name = map_entity_to_field(entity['label'])
                                new_record[field_name] = entity['text']

                        processed_records.append(Row(**new_record))

                    except Exception as e:
                        logger.error(f"Error processing record in partition {partition_id}: {str(e)}")
                        # Add record with all fields (initialized to None)
                        processed_records.append(Row(**new_record))

                for record in processed_records:
                    yield record

                current_batch = []

        # Process remaining records
        if current_batch:
            for record in current_batch:
                try:
                    new_record = {field.name: None for field in processed_schema.fields}
                    input_dict = record.asDict()

                    for key, value in input_dict.items():
                        if key in new_record:
                            new_record[key] = value

                    text = input_dict.get('description', '')
                    if text:
                        entities1 = model1.predict_entities(text, all_labels, threshold=0.54)
                        entities2 = model2.predict_entities(text, all_labels, threshold=0.7)

                        logger.debug(f"Entities from model1: {entities1}")
                        logger.debug(f"Entities from model2: {entities2}")

                        all_entities = []
                        for entity in chain(entities1, entities2):
                            cleaned = clean_entity(entity)
                            for main_label, aliases in labels_dict.items():
                                if cleaned['label'] in aliases:
                                    cleaned['label'] = main_label
                                    all_entities.append(cleaned)
                                    break

                        logger.debug(f"Combined and cleaned entities: {all_entities}")

                        entity_dict = {}
                        for entity in sorted(all_entities, key=lambda x: (-x['score'], x['label'])):
                            if entity['label'] not in entity_dict:
                                entity_dict[entity['label']] = entity

                        logger.debug(f"Final entity dictionary: {entity_dict}")

                        for entity in entity_dict.values():
                            field_name = map_entity_to_field(entity['label'])
                            new_record[field_name] = entity['text']

                    yield Row(**new_record)

                except Exception as e:
                    logger.error(f"Error processing record in final batch (partition {partition_id}): {str(e)}")
                    yield Row(**new_record)

    except Exception as e:
        logger.error(f"Major error in partition {partition_id}: {str(e)}")
        raise

In [ ]:
def clean_entity(entity):
    """Clean and normalize entity text."""
    try:
        if entity['label'] in ["year started", "incorporated year", "founded year"]:
            # Extract valid year
            match = re.search(r'\b(18|19|20)\d{2}\b', entity['text'])
            return {'label': entity['label'], 'text': match.group(0) if match else '', 'score': entity['score']}
        elif entity['label'] not in ["brand", "logo", "trade mark"]:
            # Remove non-alphanumeric characters for other labels
            cleaned_text = re.sub(r'[^a-zA-Z\s]', '', entity['text'])
            return {'label': entity['label'], 'text': cleaned_text, 'score': entity['score']}
        else:
            return entity
    except Exception as e:
        logger.error(f"Error cleaning entity: {e}")
        return entity

Without Broadcasting Model

In [ ]:
# without broadcast

def extract_ner_entities(iterator):
    """Process each partition with NER extraction using batching."""
    from pyspark.taskcontext import TaskContext
    from itertools import chain
    import traceback

    partition_id = TaskContext.get().partitionId()
    logging.info(f"Partition {partition_id}: Starting processing")

    # Initialize counters for logging
    count_in = 0
    count_out = 0
    count_errors = 0

    # Batch size for processing
    batch_size = 100
    current_batch = []

    try:
        # Decompress models and labels once per partition
        model1 = pickle.loads(zlib.decompress(broadcast_model1.value))
        model2 = pickle.loads(zlib.decompress(broadcast_model2.value))
        labels_list = broadcast_labels.value

        for row in iterator:
            count_in += 1
            current_batch.append(row)

            # Process the batch when it reaches the batch size
            if len(current_batch) >= batch_size:
                batch_results = process_batch(
                    current_batch, model1, model2, labels_list, partition_id
                )
                for result in batch_results:
                    count_out += 1
                    yield result
                current_batch = []

        # Process any remaining rows in the final batch
        if current_batch:
            batch_results = process_batch(
                current_batch, model1, model2, labels_list, partition_id
            )
            for result in batch_results:
                count_out += 1
                yield result

    except Exception as e:
        logging.error(f"Partition {partition_id}: Major error: {str(e)}")
        logging.error(traceback.format_exc())
        raise

    logging.info(
        f"Partition {partition_id}: Completed processing {count_in} rows, "
        f"generated {count_out} results, encountered {count_errors} errors"
    )


def process_batch(batch, model1, model2, labels_list, partition_id):
    """Process a batch of rows."""
    results = []

    for row in batch:
        try:
            # Convert row to dictionary for easier access
            row_dict = row.asDict()

            # Skip rows with missing or empty descriptions
            if 'description' not in row_dict or not row_dict['description']:
                logging.warning(f"Partition {partition_id}: Missing or empty description")
                continue

            # Initialize feature holders
            ner_features = {}
            entity_sets = {}

            # Extract entities from both models
            with torch.no_grad():
                entities1 = model1.predict_entities(row_dict['description'], labels_list, threshold=0.54) or []
                entities2 = model2.predict_entities(row_dict['description'], labels_list, threshold=0.7) or []

            # Combine and clean entities
            entities = sorted(
                (clean_entity(entity) for entity in chain(entities1, entities2)),
                key=lambda k: (k['label'], -k['score'])
            )

            # Group entities by label and deduplicate
            for entity in entities:
                label = entity["label"]
                text = entity["text"]
                if not text:
                    continue

                # Clean text based on label
                if label == "brand":
                    text = text.replace('brands', '').replace('brand', '').strip()
                elif label == "industry":
                    text = text.lower().replace('industry', '').strip()
                elif label == "product":
                    text = text.lower().replace('products', '').replace('product', '').strip()

                if label not in entity_sets:
                    entity_sets[label] = set()
                entity_sets[label].add(text)

                if label == "year started" and "year_started" not in ner_features:
                    ner_features["year_started"] = text

            # Concatenate values for each field
            fields = [
                "year_started", "product", "brand", "business_categories",
                "business_sub_categories", "industry", "business_services", "offer"
            ]
            for field in fields:
                ner_features[field] = " | ".join(entity_sets.get(field, []))

            # Ensure all required fields are present
            for field in fields:
                if field not in ner_features:
                    ner_features[field] = ""

            # Build the output Row object
            result = Row(
                company_id=row_dict.get('company_id', ''),
                es_ecid=row_dict.get('es_ecid', ''),
                description=row_dict.get('description', ''),
                industry_primary=row_dict.get('industry_primary', ''),
                industries=row_dict.get('industries', ''),
                sub_industries=row_dict.get('sub_industries', ''),
                top3_industries=row_dict.get('top3_industries', ''),
                year_started=ner_features.get("year_started", ""),
                product=ner_features.get("product", ""),
                brand=ner_features.get("brand", ""),
                industry=ner_features.get("industry", ""),
                business_categories=ner_features.get("business_categories", ""),
                business_sub_categories=ner_features.get("business_sub_categories", ""),
                business_services=ner_features.get("business_services", ""),
                offer=ner_features.get("offer", "")
            )

            results.append(result)

        except Exception as e:
            logging.error(f"Partition {partition_id}: Error processing row: {str(e)}")
            logging.error(traceback.format_exc())
            continue

    return results

Teradata Upload   3_9

In [ ]:
import logging
import datetime
from pyspark.taskcontext import TaskContext
from typing import Iterator
import teradatasql

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Configuration
hostname = "your_hostname"
user_id = "your_user"
password = "your_password"
logmech = "your_logmech"
table_name = "your_table"
batch_size = 1000
num_partitions = 100  # Adjust based on data size

# Initialize counters
total_rows_processed = 0
total_rows_failed = 0
start_time = datetime.datetime.now()

# Repartition the DataFrame for optimal processing
optimized_df = processed_df.repartition(num_partitions)

# Convert to RDD of tuples for efficient processing
row_rdd = optimized_df.rdd.map(tuple)

def upload_partition_to_teradata(partition_iter: Iterator[tuple]) -> Iterator[str]:
    """Process and upload a partition of data to Teradata"""
    partition_id = TaskContext.get().partitionId()
    partition_rows_processed = 0
    partition_rows_failed = 0

    # Convert iterator to list for length calculation and multiple passes
    rows = list(partition_iter)

    if not rows:
        logging.warning(f"Partition {partition_id}: Empty partition")
        return

    logging.info(f"Partition {partition_id}: Starting upload of {len(rows)} rows")

    try:
        with teradatasql.connect(host=hostname, user=user_id, password=password, logmech=logmech) as con:
            cursor = con.cursor()

            insert_sql = f"""
            INSERT INTO {table_name} (
                zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
                industries, sub_industries, top3_industries, year_started, product,
                brand, industry, business_categories, business_sub_categories,
                business_services, offer
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            total_batches = (len(rows) - 1) // batch_size + 1

            for batch_num, i in enumerate(range(0, len(rows), batch_size), 1):
                batch = rows[i:i + batch_size]
                try:
                    cursor.executemany(insert_sql, batch)
                    partition_rows_processed += len(batch)
                    logging.info(f"Partition {partition_id}: Completed batch {batch_num}/{total_batches} "
                               f"({len(batch)} rows)")
                except Exception as e:
                    partition_rows_failed += len(batch)
                    logging.error(f"Partition {partition_id}: Batch {batch_num} failed: {str(e)}")
                    continue

    except Exception as e:
        logging.error(f"Partition {partition_id}: Connection error: {str(e)}")
        partition_rows_failed += len(rows)

    # Return statistics for this partition
    return [{
        'partition_id': partition_id,
        'rows_processed': partition_rows_processed,
        'rows_failed': partition_rows_failed
    }]

# Execute the upload process
results = row_rdd.mapPartitions(upload_partition_to_teradata).collect()

# Aggregate results
total_rows_processed = sum(r['rows_processed'] for result in results for r in result)
total_rows_failed = sum(r['rows_failed'] for result in results for r in result)
end_time = datetime.datetime.now()
duration = end_time - start_time

# Print final summary
print("\nUpload Summary:")
print(f"Total rows processed successfully: {total_rows_processed:,}")
print(f"Total rows failed: {total_rows_failed:,}")
print(f"Total duration: {duration}")
print(f"Average throughput: {total_rows_processed/duration.total_seconds():,.2f} rows/second")import logging
import datetime
from pyspark.taskcontext import TaskContext
from typing import Iterator
import teradatasql

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Configuration
hostname = "your_hostname"
user_id = "your_user"
password = "your_password"
logmech = "your_logmech"
table_name = "your_table"
batch_size = 1000
num_partitions = 100  # Adjust based on data size

# Initialize counters
total_rows_processed = 0
total_rows_failed = 0
start_time = datetime.datetime.now()

# Repartition the DataFrame for optimal processing
optimized_df = processed_df.repartition(num_partitions)

# Convert to RDD of tuples for efficient processing
row_rdd = optimized_df.rdd.map(tuple)

def upload_partition_to_teradata(partition_iter: Iterator[tuple]) -> Iterator[str]:
    """Process and upload a partition of data to Teradata"""
    partition_id = TaskContext.get().partitionId()
    partition_rows_processed = 0
    partition_rows_failed = 0

    # Convert iterator to list for length calculation and multiple passes
    rows = list(partition_iter)

    if not rows:
        logging.warning(f"Partition {partition_id}: Empty partition")
        return

    logging.info(f"Partition {partition_id}: Starting upload of {len(rows)} rows")

    try:
        with teradatasql.connect(host=hostname, user=user_id, password=password, logmech=logmech) as con:
            cursor = con.cursor()

            insert_sql = f"""
            INSERT INTO {table_name} (
                zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
                industries, sub_industries, top3_industries, year_started, product,
                brand, industry, business_categories, business_sub_categories,
                business_services, offer
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            total_batches = (len(rows) - 1) // batch_size + 1

            for batch_num, i in enumerate(range(0, len(rows), batch_size), 1):
                batch = rows[i:i + batch_size]
                try:
                    cursor.executemany(insert_sql, batch)
                    partition_rows_processed += len(batch)
                    logging.info(f"Partition {partition_id}: Completed batch {batch_num}/{total_batches} "
                               f"({len(batch)} rows)")
                except Exception as e:
                    partition_rows_failed += len(batch)
                    logging.error(f"Partition {partition_id}: Batch {batch_num} failed: {str(e)}")
                    continue

    except Exception as e:
        logging.error(f"Partition {partition_id}: Connection error: {str(e)}")
        partition_rows_failed += len(rows)

    # Return statistics for this partition
    return [{
        'partition_id': partition_id,
        'rows_processed': partition_rows_processed,
        'rows_failed': partition_rows_failed
    }]

# Execute the upload process
results = row_rdd.mapPartitions(upload_partition_to_teradata).collect()

# Aggregate results
total_rows_processed = sum(r['rows_processed'] for result in results for r in result)
total_rows_failed = sum(r['rows_failed'] for result in results for r in result)
end_time = datetime.datetime.now()
duration = end_time - start_time

# Print final summary
print("\nUpload Summary:")
print(f"Total rows processed successfully: {total_rows_processed:,}")
print(f"Total rows failed: {total_rows_failed:,}")
print(f"Total duration: {duration}")
print(f"Average throughput: {total_rows_processed/duration.total_seconds():,.2f} rows/second")


Teradata Upload with Partition Debug

In [ ]:
def upload_partition_to_teradata(partition_iter: Iterator[tuple]) -> Iterator[dict]:
    """Process and upload a partition of data to Teradata."""
    from pyspark.taskcontext import TaskContext
    import teradatasql  # Ensure teradatasql is imported here

    partition_id = TaskContext.get().partitionId()
    partition_rows_processed = 0
    partition_rows_failed = 0

    # Convert iterator to list for length calculation and multiple passes
    rows = list(partition_iter)

    if not rows:
        logging.warning(f"Partition {partition_id}: Empty partition")
        return iter([])

    logging.info(f"Partition {partition_id}: Starting upload of {len(rows)} rows")

    try:
        # Establish connection to Teradata
        with teradatasql.connect(
            host=hostname,
            user=user_id,
            password=password,
            logmech=logmech
        ) as con:
            cursor = con.cursor()

            # Define the SQL insert statement
            insert_sql = f"""
            INSERT INTO {table_name} (
                zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
                industries, sub_industries, top3_industries, year_started, product,
                brand, industry, business_categories, business_sub_categories,
                business_services, offer
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            # Process rows in batches
            total_batches = (len(rows) - 1) // batch_size + 1

            for batch_num, i in enumerate(range(0, len(rows), batch_size), 1):
                batch = rows[i:i + batch_size]
                try:
                    # Convert batch elements to tuples before executing
                    batch_tuples = [tuple(row) for row in batch]
                    cursor.executemany(insert_sql, batch_tuples)
                    partition_rows_processed += len(batch)
                    logging.info(f"Partition {partition_id}: Completed batch {batch_num}/{total_batches} "
                                 f"({len(batch)} rows)")
                except Exception as e:
                    partition_rows_failed += len(batch)
                    logging.error(f"Partition {partition_id}: Batch {batch_num} failed: {str(e)}")
                    continue

    except Exception as e:
        logging.error(f"Partition {partition_id}: Connection error: {str(e)}")
        partition_rows_failed += len(rows)

    # Return statistics for this partition
    return iter([{
        'partition_id': partition_id,
        'rows_processed': partition_rows_processed,
        'rows_failed': partition_rows_failed
    }])

In [ ]:
# Create a single sample record
sample_data = [(
    '12345', 'ECID456', 'Sample Corp', 'Technology',
    'Tech,Software', 'Cloud,Data', 'Tech,AI,Cloud', '2020',
    'Software', 'SampleBrand', 'Technology', 'Enterprise',
    'B2B', 'Consulting', 'Cloud Services'
)]

# Define schema explicitly
schema = StructType([
    StructField("zi_c_company_id", StringType(), True),
    StructField("zi_es_ecid", StringType(), True),
    StructField("zi_c_description", StringType(), True),
    StructField("zi_industry_primary", StringType(), True),
    StructField("industries", StringType(), True),
    StructField("sub_industries", StringType(), True),
    StructField("top3_industries", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

# Create DataFrame
sample_df = spark.createDataFrame(sample_data, schema=schema)

# Convert to RDD
sample_rdd = sample_df.rdd.map(tuple)

# Test the upload function
results = sample_rdd.mapPartitions(upload_partition_to_teradata).collect()

# Print results
print("Results:", results)

In [ ]:
import logging
import sys
from datetime import datetime
import traceback
from pyspark.sql.types import StructType, StructField, StringType
import teradatasql

# Basic logging setup for notebook
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)

print("Starting test with sample data...")

# Create sample data
sample_data = [(
    '12345', 'ECID456', 'Sample Corp', 'Technology',
    'Tech,Software', 'Cloud,Data', 'Tech,AI,Cloud', '2020',
    'Software', 'SampleBrand', 'Technology', 'Enterprise',
    'B2B', 'Consulting', 'Cloud Services'
)]

print(f"Sample data created: {sample_data}")

# Create DataFrame with schema
schema = StructType([
    StructField("zi_c_company_id", StringType(), True),
    StructField("zi_es_ecid", StringType(), True),
    StructField("zi_c_description", StringType(), True),
    StructField("zi_industry_primary", StringType(), True),
    StructField("industries", StringType(), True),
    StructField("sub_industries", StringType(), True),
    StructField("top3_industries", StringType(), True),
    StructField("year_started", StringType(), True),
    StructField("product", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("business_categories", StringType(), True),
    StructField("business_sub_categories", StringType(), True),
    StructField("business_services", StringType(), True),
    StructField("offer", StringType(), True)
])

sample_df = spark.createDataFrame(sample_data, schema=schema)
print("DataFrame created successfully")

# Convert to RDD
sample_rdd = sample_df.rdd.map(tuple)
print("RDD created successfully")

# Get the data from the first partition
first_partition_data = list(sample_rdd.collect())
print(f"\nData to be processed: {first_partition_data}")

try:
    print("\nAttempting to connect to Teradata...")
    with teradatasql.connect(
        host=hostname,
        user=user_id,
        password=password,
        logmech=logmech
    ) as con:
        print("Successfully connected to Teradata")
        cursor = con.cursor()

        insert_sql = f"""
        INSERT INTO {table_name} (
            zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
            industries, sub_industries, top3_industries, year_started, product,
            brand, industry, business_categories, business_sub_categories,
            business_services, offer
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        rows_processed = 0
        rows_failed = 0
        error_messages = []

        for row in first_partition_data:
            try:
                print(f"\nProcessing row: {row}")
                cursor.execute(insert_sql, row)
                rows_processed += 1
                print(f"Successfully processed row")

            except Exception as row_error:
                rows_failed += 1
                error_msg = f"Row processing error: {str(row_error)}\n{traceback.format_exc()}"
                error_messages.append(error_msg)
                print(f"Error processing row: {error_msg}")

except Exception as conn_error:
    error_msg = f"Connection error: {str(conn_error)}\n{traceback.format_exc()}"
    print(f"Connection error occurred: {error_msg}")
    rows_failed = len(first_partition_data)

# Print final results
print("\n=== Upload Results ===")
print(f"Rows processed: {rows_processed}")
print(f"Rows failed: {rows_failed}")
if error_messages:
    print("\nError Messages:")
    for msg in error_messages:
        print(f"\n{msg}")

In [ ]:
def upload_partition_to_teradata(partition_iter):
    """Process and upload a partition of data to Teradata."""
    from pyspark.taskcontext import TaskContext
    import teradatasql
    import traceback

    partition_id = TaskContext.get().partitionId()
    rows_processed = 0
    rows_failed = 0
    error_messages = []

    # Convert iterator to list for easier debugging
    rows = list(partition_iter)

    if not rows:
        print(f"Partition {partition_id}: Empty partition")
        return iter([{
            'partition_id': partition_id,
            'rows_processed': 0,
            'rows_failed': 0,
            'error_messages': ['Empty partition']
        }])

    print(f"Partition {partition_id}: Starting upload of {len(rows)} rows")

    try:
        # Connect to Teradata
        with teradatasql.connect(
            host=hostname,
            user=user_id,
            password=password,
            logmech=logmech
        ) as con:
            print(f"Partition {partition_id}: Successfully connected to Teradata")
            cursor = con.cursor()

            # Updated SQL insert statement with the new field list
            insert_sql = f"""
            INSERT INTO {table_name} (
                zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
                zi_industries, zi_sub_industries, top3_industries, year_started, product,
                brand, industry, business_categories, business_sub_categories,
                business_services, offer
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            for row in rows:
                try:
                    print(f"Partition {partition_id}: Processing row: {row}")
                    cursor.execute(insert_sql, row)
                    rows_processed += 1
                except Exception as row_error:
                    rows_failed += 1
                    error_msg = f"Row processing error: {str(row_error)}\n{traceback.format_exc()}"
                    error_messages.append(error_msg)
                    print(f"Partition {partition_id}: Error processing row: {error_msg}")

    except Exception as conn_error:
        error_msg = f"Connection error: {str(conn_error)}\n{traceback.format_exc()}"
        error_messages.append(error_msg)
        print(f"Partition {partition_id}: Connection error occurred: {error_msg}")
        rows_failed = len(rows)

    # Return results for this partition
    return iter([{
        'partition_id': partition_id,
        'rows_processed': rows_processed,
        'rows_failed': rows_failed,
        'error_messages': error_messages
    }])

# Use the function in your Spark pipeline
results = sample_rdd.mapPartitions(upload_partition_to_teradata).collect()

# Print final results
print("\n=== Final Upload Results ===")
for result in results:
    print(f"\nPartition {result['partition_id']}:")
    print(f"Rows processed: {result['rows_processed']}")
    print(f"Rows failed: {result['rows_failed']}")
    if result['error_messages']:
        print("\nError Messages:")
        for msg in result['error_messages']:
            print(f"\n{msg}")

In [ ]:
import teradatasql
import traceback

def process_dataframe(spark_df, batch_size=1000):
    """Process the full DataFrame in batches"""
    print("Starting data processing...")

    # Convert Spark DataFrame to list of tuples for processing
    data_to_process = spark_df.rdd.map(tuple).collect()
    total_rows = len(data_to_process)

    print(f"Total rows to process: {total_rows}")

    rows_processed = 0
    rows_failed = 0
    error_messages = []

    try:
        print("Attempting to connect to Teradata...")
        with teradatasql.connect(
            host=hostname,
            user=user_id,
            password=password,
            logmech=logmech
        ) as con:
            print("Successfully connected to Teradata")
            cursor = con.cursor()

            insert_sql = f"""
            INSERT INTO {table_name} (
                zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
                zi_industries, zi_sub_industries, top3_industries, year_started, product,
                brand, industry, business_categories, business_sub_categories,
                business_services, offer
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            # Process in batches
            for i in range(0, total_rows, batch_size):
                batch = data_to_process[i:i + batch_size]
                print(f"\nProcessing batch {i//batch_size + 1} of {(total_rows + batch_size - 1)//batch_size}")

                for row in batch:
                    try:
                        cursor.execute(insert_sql, row)
                        rows_processed += 1

                        # Print progress every 100 rows
                        if rows_processed % 100 == 0:
                            print(f"Processed {rows_processed} rows out of {total_rows}")

                    except Exception as row_error:
                        rows_failed += 1
                        error_msg = (
                            f"Row processing error: {str(row_error)}\n"
                            f"Row data: {row}\n"
                        )
                        error_messages.append(error_msg)
                        print(f"Error processing row: {error_msg}")

                # Commit after each batch
                con.commit()
                print(f"Committed batch {i//batch_size + 1}")

    except Exception as conn_error:
        error_msg = f"Connection error: {str(conn_error)}"
        error_messages.append(error_msg)
        print(f"Connection error occurred: {error_msg}")
        rows_failed = total_rows - rows_processed

    # Print final results
    print("\n=== Final Upload Results ===")
    print(f"Total rows processed: {rows_processed}")
    print(f"Total rows failed: {rows_failed}")
    if error_messages:
        print("\nError Messages:")
        for msg in error_messages:
            print(f"\n{msg}")

    return {
        'rows_processed': rows_processed,
        'rows_failed': rows_failed,
        'error_messages': error_messages
    }

# Usage example:
try:
    # Process your full DataFrame
    results = process_dataframe(your_spark_df, batch_size=1000)

    # You can access the results if needed
    print(f"\nFinal Summary:")
    print(f"Successfully processed: {results['rows_processed']} rows")
    print(f"Failed: {results['rows_failed']} rows")

except Exception as e:
    print(f"An error occurred during processing: {str(e)}")

In [ ]:
# Example usage with your full DataFrame
results = process_dataframe(your_spark_df, batch_size=1000)

# Or if you want to try with a smaller batch size
results = process_dataframe(your_spark_df, batch_size=100)

In [ ]:
import teradatasql
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from queue import Queue

def process_batch(batch_data, connection_params, insert_sql, batch_num, total_batches):
    """Process a single batch of data using its own connection"""
    results = {
        'processed': 0,
        'failed': 0,
        'errors': []
    }

    try:
        with teradatasql.connect(**connection_params) as con:
            cursor = con.cursor()

            # Use executemany instead of execute for batch insertion
            try:
                cursor.executemany(insert_sql, batch_data)
                con.commit()
                results['processed'] = len(batch_data)
                print(f"Batch {batch_num}/{total_batches}: Successfully processed {len(batch_data)} rows")

            except Exception as batch_error:
                # If batch insert fails, fall back to row-by-row
                print(f"Batch {batch_num}/{total_batches}: Batch insert failed, falling back to row-by-row")
                for row in batch_data:
                    try:
                        cursor.execute(insert_sql, row)
                        results['processed'] += 1
                    except Exception as row_error:
                        results['failed'] += 1
                        results['errors'].append(f"Row error: {str(row_error)}")
                con.commit()

    except Exception as conn_error:
        results['failed'] = len(batch_data)
        results['errors'].append(f"Connection error in batch {batch_num}: {str(conn_error)}")

    return results

def process_dataframe_parallel(spark_df, batch_size=1000, max_workers=4):
    """Process the DataFrame in parallel batches"""
    print("Starting parallel data processing...")

    # Convert Spark DataFrame to list of tuples
    data_to_process = spark_df.rdd.map(tuple).collect()
    total_rows = len(data_to_process)

    # Create batches
    batches = [
        data_to_process[i:i + batch_size]
        for i in range(0, total_rows, batch_size)
    ]
    total_batches = len(batches)

    print(f"Total rows: {total_rows}")
    print(f"Number of batches: {total_batches}")
    print(f"Workers: {max_workers}")

    # Connection parameters
    connection_params = {
        'host': hostname,
        'user': user_id,
        'password': password,
        'logmech': logmech
    }

    # SQL statement
    insert_sql = f"""
    INSERT INTO {table_name} (
        zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
        zi_industries, zi_sub_industries, top3_industries, year_started, product,
        brand, industry, business_categories, business_sub_categories,
        business_services, offer
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    total_processed = 0
    total_failed = 0
    all_errors = []

    # Process batches in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all batch processing tasks
        future_to_batch = {
            executor.submit(
                process_batch,
                batch,
                connection_params,
                insert_sql,
                i+1,
                total_batches
            ): i
            for i, batch in enumerate(batches)
        }

        # Process completed batches
        for future in as_completed(future_to_batch):
            batch_result = future.result()
            total_processed += batch_result['processed']
            total_failed += batch_result['failed']
            all_errors.extend(batch_result['errors'])

            # Print progress
            progress = (total_processed + total_failed) / total_rows * 100
            print(f"Progress: {progress:.2f}% ({total_processed + total_failed}/{total_rows} rows)")

    # Print final results
    print("\n=== Final Upload Results ===")
    print(f"Total rows processed: {total_processed}")
    print(f"Total rows failed: {total_failed}")
    if all_errors:
        print("\nErrors encountered:")
        for error in all_errors:
            print(error)

    return {
        'rows_processed': total_processed,
        'rows_failed': total_failed,
        'errors': all_errors
    }

# Usage example:
try:
    # Process your full DataFrame with parallel processing
    results = process_dataframe_parallel(
        your_spark_df,
        batch_size=1000,  # Adjust based on your data
        max_workers=4     # Adjust based on your system
    )

except Exception as e:
    print(f"An error occurred during processing: {str(e)}")



## Steps to use
# For smaller datasets or less powerful systems
results = process_dataframe_parallel(your_spark_df, batch_size=500, max_workers=2)

# For larger datasets on powerful systems
results = process_dataframe_parallel(your_spark_df, batch_size=2000, max_workers=8)


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import SparkSession
import teradatasql
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import time

# Create sample data
def create_sample_data(num_rows=100):
    data = []
    for i in range(num_rows):
        row = (
            f"COMP_{i}",                    # zi_c_company_id
            f"ECID_{i}",                    # zi_es_ecid
            f"Company Description {i}",      # zi_c_description
            f"Industry {i%5}",              # zi_industry_primary
            f"Industries {i%3}",            # zi_industries
            f"Sub Industries {i%4}",        # zi_sub_industries
            f"Top Industries {i%3}",        # zi_top3_industries
            str(2000 + (i%20)),            # year_started
            f"Product {i%5}",               # product
            f"Brand {i%4}",                 # brand
            f"Industry Type {i%3}",         # industry
            f"Category {i%5}",              # business_categories
            f"Sub Category {i%4}",          # business_sub_categories
            f"Service {i%3}",               # business_services
            f"Offer {i%4}"                  # offer
        )
        data.append(row)
    return data

# Create Spark DataFrame
def create_spark_df(spark, data):
    schema = StructType([
        StructField("zi_c_company_id", StringType(), True),
        StructField("zi_es_ecid", StringType(), True),
        StructField("zi_c_description", StringType(), True),
        StructField("zi_industry_primary", StringType(), True),
        StructField("zi_industries", StringType(), True),
        StructField("zi_sub_industries", StringType(), True),
        StructField("zi_top3_industries", StringType(), True),
        StructField("year_started", StringType(), True),
        StructField("product", StringType(), True),
        StructField("brand", StringType(), True),
        StructField("industry", StringType(), True),
        StructField("business_categories", StringType(), True),
        StructField("business_sub_categories", StringType(), True),
        StructField("business_services", StringType(), True),
        StructField("offer", StringType(), True)
    ])
    return spark.createDataFrame(data, schema)

def process_batch(batch_data, connection_params, insert_sql, batch_num, total_batches):
    """Process a single batch of data using its own connection"""
    results = {
        'processed': 0,
        'failed': 0,
        'errors': [],
        'messages': []
    }

    batch_start_time = time.time()
    results['messages'].append(f"\nBatch {batch_num}/{total_batches}: Starting processing of {len(batch_data)} rows")

    try:
        results['messages'].append(f"Batch {batch_num}: Attempting to connect to Teradata...")
        with teradatasql.connect(**connection_params) as con:
            results['messages'].append(f"Batch {batch_num}: Successfully connected to Teradata")
            cursor = con.cursor()

            # Use executemany instead of execute for batch insertion
            try:
                results['messages'].append(f"Batch {batch_num}: Attempting batch insert of {len(batch_data)} rows")
                cursor.executemany(insert_sql, batch_data)
                results['messages'].append(f"Batch {batch_num}: Batch insert successful")

                results['messages'].append(f"Batch {batch_num}: Attempting to commit...")
                con.commit()
                results['messages'].append(f"Batch {batch_num}: Commit successful")

                results['processed'] = len(batch_data)

            except Exception as batch_error:
                results['messages'].append(f"Batch {batch_num}: Batch insert failed, error: {str(batch_error)}")
                results['messages'].append(f"Batch {batch_num}: Falling back to row-by-row processing")

                # If batch insert fails, fall back to row-by-row
                for idx, row in enumerate(batch_data):
                    try:
                        cursor.execute(insert_sql, row)
                        results['processed'] += 1
                        if idx % 10 == 0:  # Log every 10 rows
                            results['messages'].append(f"Batch {batch_num}: Processed {idx+1}/{len(batch_data)} rows")
                    except Exception as row_error:
                        results['failed'] += 1
                        error_msg = f"Row {idx} error in batch {batch_num}: {str(row_error)}"
                        results['errors'].append(error_msg)
                        results['messages'].append(error_msg)

                results['messages'].append(f"Batch {batch_num}: Attempting final commit...")
                con.commit()
                results['messages'].append(f"Batch {batch_num}: Final commit successful")

    except Exception as conn_error:
        results['failed'] = len(batch_data)
        error_msg = f"Connection error in batch {batch_num}: {str(conn_error)}\n{traceback.format_exc()}"
        results['errors'].append(error_msg)
        results['messages'].append(error_msg)

    batch_end_time = time.time()
    results['messages'].append(
        f"Batch {batch_num}: Completed in {batch_end_time - batch_start_time:.2f} seconds. "
        f"Processed: {results['processed']}, Failed: {results['failed']}"
    )

    return results

def process_dataframe_parallel(spark_df, batch_size=1000, max_workers=4):
    """Process the DataFrame in parallel batches"""
    print("\n=== Starting Parallel Data Processing ===")
    start_time = time.time()

    # Convert Spark DataFrame to list of tuples
    print("Converting Spark DataFrame to list of tuples...")
    data_to_process = spark_df.rdd.map(tuple).collect()
    total_rows = len(data_to_process)

    # Create batches
    batches = [
        data_to_process[i:i + batch_size]
        for i in range(0, total_rows, batch_size)
    ]
    total_batches = len(batches)

    print(f"\nInitial Setup:")
    print(f"Total rows: {total_rows}")
    print(f"Batch size: {batch_size}")
    print(f"Number of batches: {total_batches}")
    print(f"Number of workers: {max_workers}")

    # Connection parameters
    connection_params = {
        'host': hostname,
        'user': user_id,
        'password': password,
        'logmech': logmech
    }

    # SQL statement
    insert_sql = f"""
    INSERT INTO {table_name} (
        zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
        zi_industries, zi_sub_industries, zi_top3_industries, year_started, product,
        brand, industry, business_categories, business_sub_categories,
        business_services, offer
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    print("\nSQL Statement:")
    print(insert_sql)

    total_processed = 0
    total_failed = 0
    all_errors = []
    all_messages = []

    # Process batches in parallel using ThreadPoolExecutor
    print("\n=== Starting Batch Processing ===")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all batch processing tasks
        future_to_batch = {
            executor.submit(
                process_batch,
                batch,
                connection_params,
                insert_sql,
                i+1,
                total_batches
            ): i
            for i, batch in enumerate(batches)
        }

        # Process completed batches
        for future in as_completed(future_to_batch):
            batch_result = future.result()
            total_processed += batch_result['processed']
            total_failed += batch_result['failed']
            all_errors.extend(batch_result['errors'])
            all_messages.extend(batch_result['messages'])

            # Print batch messages immediately
            for message in batch_result['messages']:
                print(message)

            # Print progress
            progress = (total_processed + total_failed) / total_rows * 100
            print(f"Overall Progress: {progress:.2f}% ({total_processed + total_failed}/{total_rows} rows)")

    end_time = time.time()
    total_time = end_time - start_time

    # Print final results
    print("\n=== Final Upload Results ===")
    print(f"Total time: {total_time:.2f} seconds")
    print(f"Total rows processed: {total_processed}")
    print(f"Total rows failed: {total_failed}")
    print(f"Average processing rate: {total_rows/total_time:.2f} rows/second")

    if all_errors:
        print("\nErrors encountered:")
        for error in all_errors:
            print(error)

    return {
        'rows_processed': total_processed,
        'rows_failed': total_failed,
        'errors': all_errors,
        'messages': all_messages,
        'total_time': total_time
    }

# Create Spark session and test the code
spark = SparkSession.builder.appName("TeradataUpload").getOrCreate()

# Create sample data and DataFrame
sample_data = create_sample_data(num_rows=1000)  # Create 1000 sample rows
sample_df = create_spark_df(spark, sample_data)

# Process the DataFrame
try:
    print("Starting data upload process...")
    results = process_dataframe_parallel(
        sample_df,
        batch_size=100,    # Process 100 rows per batch
        max_workers=4      # Use 4 parallel workers
    )

    print("\n=== Execution Summary ===")
    print(f"Total execution time: {results['total_time']:.2f} seconds")
    print(f"Rows processed successfully: {results['rows_processed']}")
    print(f"Rows failed: {results['rows_failed']}")

    if results['errors']:
        print("\nErrors encountered during processing:")
        for error in results['errors']:
            print(error)
    else:
        print("\nNo errors encountered during processing")

except Exception as e:
    print(f"An error occurred during processing: {str(e)}")
    print(f"Traceback: {traceback.format_exc()}")

finally:
    spark.stop()

# upload rdd parallel

In [ ]:
def process_rdd_parallel(rdd, batch_size=1000, max_workers=4):
    """Process the RDD in parallel batches"""
    print("\n=== Starting Parallel Data Processing ===")
    start_time = time.time()

    # Convert RDD to partitions (each partition is already a list of tuples)
    data_partitions = rdd.glom().collect()
    total_rows = rdd.count()
    total_batches = len(data_partitions)

    print(f"\nInitial Setup:")
    print(f"Total rows: {total_rows}")
    print(f"Number of partitions: {total_batches}")
    print(f"Number of workers: {max_workers}")

    # Connection parameters
    connection_params = {
        'host': hostname,
        'user': user_id,
        'password': password,
        'logmech': logmech
    }

    # SQL statement
    insert_sql = f"""
    INSERT INTO {table_name} (
        zi_c_company_id, zi_es_ecid, zi_c_description, zi_industry_primary,
        zi_industries, zi_sub_industries, zi_top3_industries, year_started, product,
        brand, industry, business_categories, business_sub_categories,
        business_services, offer
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    print("\nSQL Statement:")
    print(insert_sql)

    total_processed = 0
    total_failed = 0
    all_errors = []
    all_messages = []

    # Process partitions in parallel using ThreadPoolExecutor
    print("\n=== Starting Partition Processing ===")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all partition processing tasks
        future_to_partition = {
            executor.submit(
                process_batch,
                partition,
                connection_params,
                insert_sql,
                i+1,
                total_batches
            ): i
            for i, partition in enumerate(data_partitions)
        }

        # Process completed partitions
        for future in as_completed(future_to_partition):
            partition_result = future.result()
            total_processed += partition_result['processed']
            total_failed += partition_result['failed']
            all_errors.extend(partition_result['errors'])
            all_messages.extend(partition_result['messages'])

            # Print partition messages immediately
            for message in partition_result['messages']:
                print(message)

            # Print progress
            progress = (total_processed + total_failed) / total_rows * 100
            print(f"Overall Progress: {progress:.2f}% ({total_processed + total_failed}/{total_rows} rows)")

    end_time = time.time()
    total_time = end_time - start_time

    # Print final results
    print("\n=== Final Upload Results ===")
    print(f"Total time: {total_time:.2f} seconds")
    print(f"Total rows processed: {total_processed}")
    print(f"Total rows failed: {total_failed}")
    print(f"Average processing rate: {total_rows/total_time:.2f} rows/second")

    if all_errors:
        print("\nErrors encountered:")
        for error in all_errors:
            print(error)

    return {
        'rows_processed': total_processed,
        'rows_failed': total_failed,
        'errors': all_errors,
        'messages': all_messages,
        'total_time': total_time
    }

# Usage example:
try:
    print("Starting data upload process...")
    results = process_rdd_parallel(
        processed_rdd,  # Use the RDD directly
        batch_size=100,    # Process 100 rows per batch
        max_workers=4      # Use 4 parallel workers
    )

    print("\n=== Execution Summary ===")
    print(f"Total execution time: {results['total_time']:.2f} seconds")
    print(f"Rows processed successfully: {results['rows_processed']}")
    print(f"Rows failed: {results['rows_failed']}")

    if results['errors']:
        print("\nErrors encountered during processing:")
        for error in results['errors']:
            print(error)
    else:
        print("\nNo errors encountered during processing")

except Exception as e:
    print(f"An error occurred during processing: {str(e)}")
    print(f"Traceback: {traceback.format_exc()}")